In [23]:
import pandas as pd
import joblib
import numpy as np
from tqdm import tqdm

# DrugBank Positive Samples

#### Complete DrugBank

In [16]:
fulldb = joblib.load('DB_df_clean_PubChem_Fetch.pkl')

In [17]:
len(set(fulldb['drug_id'])), len(set(fulldb['drug_inchi_key']))

(7321, 7307)

In [18]:
fulldb_smiles = fulldb[['drug_name','drug_inchi_key','drug_smiles','target_name','target_aa_sequence']]

In [19]:
fulldb_smiles = fulldb_smiles.rename(columns={'drug_name':'chem_name','drug_inchi_key':'InChiKey','drug_smiles':'chem_smile','target_aa_sequence':'aaseq'})

In [20]:
fulldb_smiles = fulldb_smiles.drop_duplicates(subset=['InChiKey','aaseq'])

In [21]:
fulldb_smiles

,chem_name,InChiKey,chem_smile,target_name,aaseq
30,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Prothrombin,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...
31,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Myeloperoxidase,MGVPFFSSLRCMVDLGPCWAGGLTAEMKLLLALAGLLAILATPQPS...
32,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Gonadotropin-releasing hormone receptor,MANSASPEQNQNHCSAINNSIPLMQGNLPTLTLSGKIRVTVTFFLF...
33,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Cytochrome P450 3A4,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...
56,Goserelin,BLCLNMBMMGCOAS-URPVMXJPSA-N,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,Lutropin-choriogonadotropic hormone receptor,MKQRFSALQLLKLLLLLQPPLPRALREALCPEPCNCVPDGALRCPG...
...,...,...,...,...,...
26949,Prednisolone acetate,LRJOMUJRLNCICJ-JZYPGELDSA-N,[H][C@@]12CC[C@](O)(C(=O)COC(C)=O)[C@@]1(C)C[C...,Serum albumin,MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKAL...
26950,Prednisolone acetate,LRJOMUJRLNCICJ-JZYPGELDSA-N,[H][C@@]12CC[C@](O)(C(=O)COC(C)=O)[C@@]1(C)C[C...,Solute carrier organic anion transporter famil...,MGETEKRIETHRIRCLSKLKMFLLAITCAFVSKTLSGSYMNSMLTQ...
26951,Prednisolone acetate,LRJOMUJRLNCICJ-JZYPGELDSA-N,[H][C@@]12CC[C@](O)(C(=O)COC(C)=O)[C@@]1(C)C[C...,P-glycoprotein 1,MDLEGDRNGGAKKKNFFKLNNKSEKDKKEKKPTVSVFSMFRYSNWL...
26952,MRTX849,PEMUGDMSUDYLHU-ZEQRLZLVSA-N,CN1CCC[C@H]1COC2=NC3=C(CCN(C3)C4=CC=CC5=C4C(=C...,GTPase KRas,MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...


In [22]:
## Remove Upper Threshold Samples

res_text = open("BindingDB Curated/res_dump.txt", "r")
res_list = res_text.read().split('\n')

pairs_text = open("BindingDB Curated/uniqueDTpairs_dump.txt", "r")
pairs_list = pairs_text.read().split('\n')

In [24]:
import ast

bindingdictWholeBindingDB = dict()

for i in tqdm(range(len(pairs_list))):
    bindingdictWholeBindingDB[ast.literal_eval(pairs_list[i])] = ast.literal_eval(res_list[i])
    if ast.literal_eval(res_list[i]) is None:
        bindingdictWholeBindingDB[ast.literal_eval(pairs_list[i])] = {}

100%|█████████████████████████████████████████████████████████████████████| 1376181/1376181 [00:45<00:00, 30354.83it/s]


In [25]:
DBset={(c,t) for c,t in fulldb[['drug_inchi_key','target_aa_sequence']].values}

tr=10**6
labelinactiveBindingDB={k for k,v in bindingdictWholeBindingDB.items() if (sum(np.array(list(v.values()))>=tr)>=len(v.values()))}

In [26]:
upperthreshold = DBset&labelinactiveBindingDB
len(DBset&labelinactiveBindingDB)

53

In [27]:
upperthreshold = list(upperthreshold)

In [28]:
len(upperthreshold)

53

In [29]:
to_drop = []

for index, row in tqdm(fulldb_smiles.iterrows()):
    if ((row['InChiKey'],row['aaseq'])) in upperthreshold:
        to_drop.append(index)
        
fulldb_smiles_binding = fulldb_smiles.drop(to_drop)

25373it [00:02, 11827.03it/s]


In [30]:
fulldb_smiles_binding['Y'] = 1
fulldb_smiles_binding['is_ndm'] = 0
fulldb_smiles_binding['is_bindingDB'] = 0

In [31]:
fulldb_smiles_binding

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB
30,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Prothrombin,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...,1,0,0
31,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Myeloperoxidase,MGVPFFSSLRCMVDLGPCWAGGLTAEMKLLLALAGLLAILATPQPS...,1,0,0
32,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Gonadotropin-releasing hormone receptor,MANSASPEQNQNHCSAINNSIPLMQGNLPTLTLSGKIRVTVTFFLF...,1,0,0
33,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Cytochrome P450 3A4,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,1,0,0
56,Goserelin,BLCLNMBMMGCOAS-URPVMXJPSA-N,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,Lutropin-choriogonadotropic hormone receptor,MKQRFSALQLLKLLLLLQPPLPRALREALCPEPCNCVPDGALRCPG...,1,0,0
...,...,...,...,...,...,...,...,...
26949,Prednisolone acetate,LRJOMUJRLNCICJ-JZYPGELDSA-N,[H][C@@]12CC[C@](O)(C(=O)COC(C)=O)[C@@]1(C)C[C...,Serum albumin,MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKAL...,1,0,0
26950,Prednisolone acetate,LRJOMUJRLNCICJ-JZYPGELDSA-N,[H][C@@]12CC[C@](O)(C(=O)COC(C)=O)[C@@]1(C)C[C...,Solute carrier organic anion transporter famil...,MGETEKRIETHRIRCLSKLKMFLLAITCAFVSKTLSGSYMNSMLTQ...,1,0,0
26951,Prednisolone acetate,LRJOMUJRLNCICJ-JZYPGELDSA-N,[H][C@@]12CC[C@](O)(C(=O)COC(C)=O)[C@@]1(C)C[C...,P-glycoprotein 1,MDLEGDRNGGAKKKNFFKLNNKSEKDKKEKKPTVSVFSMFRYSNWL...,1,0,0
26952,MRTX849,PEMUGDMSUDYLHU-ZEQRLZLVSA-N,CN1CCC[C@H]1COC2=NC3=C(CCN(C3)C4=CC=CC5=C4C(=C...,GTPase KRas,MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,1,0,0


In [32]:
db_full_pairs = []

for index, row in fulldb.iterrows():
    db_full_pairs.append((row['drug_inchi_key'],row['target_aa_sequence']))

In [33]:
chem_dict = dict()
target_dict = dict()

for index, row in fulldb.iterrows():
    chem_dict[row['drug_inchi_key']] = dict()
    chem_dict[row['drug_inchi_key']]['chem_name'] = row['drug_name']
    chem_dict[row['drug_inchi_key']]['chem_smile'] = row['drug_smiles']
    target_dict[row['target_aa_sequence']] = row['target_name']

## Add SMILEs and InChIKeys from BindingDB

In [34]:
BindingDB_df = pd.read_csv("../DrugBank-BindingDB/BindingDB_All.tsv", sep='\t', error_bad_lines=False)
BindingDB_df_unique_chems = BindingDB_df.drop_duplicates(subset=['Ligand SMILES'])

b'Skipping line 772572: expected 193 fields, saw 205\nSkipping line 772598: expected 193 fields, saw 205\n'
b'Skipping line 805291: expected 193 fields, saw 205\n'
b'Skipping line 827961: expected 193 fields, saw 265\n'
b'Skipping line 1231688: expected 193 fields, saw 241\n'
b'Skipping line 1345591: expected 193 fields, saw 241\nSkipping line 1345592: expected 193 fields, saw 241\nSkipping line 1345593: expected 193 fields, saw 241\nSkipping line 1345594: expected 193 fields, saw 241\nSkipping line 1345595: expected 193 fields, saw 241\nSkipping line 1345596: expected 193 fields, saw 241\nSkipping line 1345597: expected 193 fields, saw 241\nSkipping line 1345598: expected 193 fields, saw 241\nSkipping line 1345599: expected 193 fields, saw 241\n'
b'Skipping line 1358864: expected 193 fields, saw 205\n'
b'Skipping line 1378087: expected 193 fields, saw 241\nSkipping line 1378088: expected 193 fields, saw 241\nSkipping line 1378089: expected 193 fields, saw 241\nSkipping line 1378090: e

In [35]:
for index, row in tqdm(BindingDB_df.iterrows()):
    if row['Ligand InChI Key'] not in list(chem_dict.keys()):
        chem_dict[row['Ligand InChI Key']] = dict()
        chem_dict[row['Ligand InChI Key']]['chem_name'] = row['BindingDB Ligand Name']
        chem_dict[row['Ligand InChI Key']]['chem_smile'] = row['Ligand SMILES']
    if row['BindingDB Target Chain  Sequence'] not in list(target_dict.keys()):
        target_dict[row['BindingDB Target Chain  Sequence']] = row['Target Name Assigned by Curator or DataSource']

1813527it [7:35:27, 66.36it/s]


#### DrugBank in BindingDB

In [48]:
db_pos_raw = joblib.load('bindingdictDBclean.pkl')

In [49]:
db_bd_pairs = list(set(db_pos_raw.keys()))

In [50]:
tr=10**3
labelactive=[k for k,v in db_pos_raw.items() if (sum(np.array(list(v.values()))<=tr)>0)]
len(labelactive)

5839

In [51]:
drugbank_bd_df = pd.DataFrame()

inchilist = []
aalist = []

for pair in labelactive:
    inchilist.append(pair[0])
    aalist.append(pair[1])
    
drugbank_bd_df['InChiKey'] = inchilist
drugbank_bd_df['aaseq'] = aalist

In [52]:
bd_load = BindingDB = pd.read_csv("BindingDB_All.tsv", sep='\t', error_bad_lines=False)

b'Skipping line 772572: expected 193 fields, saw 205\nSkipping line 772598: expected 193 fields, saw 205\n'
b'Skipping line 805291: expected 193 fields, saw 205\n'
b'Skipping line 827961: expected 193 fields, saw 265\n'
b'Skipping line 1231688: expected 193 fields, saw 241\n'
b'Skipping line 1345591: expected 193 fields, saw 241\nSkipping line 1345592: expected 193 fields, saw 241\nSkipping line 1345593: expected 193 fields, saw 241\nSkipping line 1345594: expected 193 fields, saw 241\nSkipping line 1345595: expected 193 fields, saw 241\nSkipping line 1345596: expected 193 fields, saw 241\nSkipping line 1345597: expected 193 fields, saw 241\nSkipping line 1345598: expected 193 fields, saw 241\nSkipping line 1345599: expected 193 fields, saw 241\n'
b'Skipping line 1358864: expected 193 fields, saw 205\n'
b'Skipping line 1378087: expected 193 fields, saw 241\nSkipping line 1378088: expected 193 fields, saw 241\nSkipping line 1378089: expected 193 fields, saw 241\nSkipping line 1378090: e

In [53]:
## Expanding the target mapping to include both DrugBank and BindingDB

bd_load = bd_load.drop_duplicates(subset=['BindingDB Target Chain  Sequence']) 

for index, row in bd_load.iterrows():
    target_dict[row['BindingDB Target Chain  Sequence']] = row['UniProt (SwissProt) Recommended Name of Target Chain']

In [54]:
## Drop chemicals which do not belong to DrugBank

to_drop_noDB = []
dropped_chems = []

for index ,row in drugbank_bd_df.iterrows():
    try:
        drugbank_bd_df.at[index,'chem_name'] = chem_dict[row['InChiKey']]['chem_name'] 
        drugbank_bd_df.at[index,'chem_smile'] = chem_dict[row['InChiKey']]['chem_smile']
        drugbank_bd_df.at[index,'target_name'] = target_dict[row['aaseq']]
    except:
        to_drop_noDB.append(index)
        dropped_chems.append(row['InChiKey'])
        
drugbank_bd_df_binding = drugbank_bd_df.drop(to_drop_noDB)

In [55]:
len(set(to_drop_noDB)), len(set(dropped_chems))

(1509, 497)

In [56]:
drugbank_bd_df_binding

,InChiKey,aaseq,chem_name,chem_smile,target_name
0,BCFGMOOMADDAQU-UHFFFAOYSA-N,MPPRPSSGELWGIHLMPPRILVECLLPNGMIVTLECLREATLITIK...,Lapatinib,CS(=O)(=O)CCNCC1=CC=C(O1)C1=CC2=C(C=C1)N=CN=C2...,"Phosphatidylinositol 4,5-bisphosphate 3-kinase..."
1,FOIVPCKZDPCJJY-JQIJEIRASA-N,MATNKERLFAAGALGPGSGYPGAGFPFAFPGALRGSPPFEMLSPSF...,Arotinoid acid,C\C(=C/C1=CC=C(C=C1)C(O)=O)C1=CC=C2C(=C1)C(C)(...,Retinoic acid receptor gamma
2,ARMFMDYRYOKSOW-UHFFFAOYSA-N,MSRSKRDNNFYSVEIGDSTFTVLKRYQNLKPIGSGAQGIVCAAYDA...,4-{[5-chloro-4-(1H-indol-3-yl)pyrimidin-2-yl]a...,CCNC(=O)N1CCC(CC1)NC1=NC(C2=CNC3=CC=CC=C23)=C(...,Mitogen-activated protein kinase 8
3,SLPJGDQJLTYWCI-UHFFFAOYSA-N,MPGPAAGSRARVYAEVNSLRSREYWDYEAHVPSWGNQDDYQLVRKL...,"DIMETHYL-(4,5,6,7-TETRABROMO-1H-BENZOIMIDAZOL-...",CN(C)C1=NC2=C(Br)C(Br)=C(Br)C(Br)=C2N1,Casein kinase II subunit alpha'
4,IYIKLHRQXLHMJQ-UHFFFAOYSA-N,MPVRRGHVAPQNTFLDTIIRKFEGQSRKFIIANARVENCAVIYCND...,Amiodarone,CCCCC1=C(C(=O)C2=CC(I)=C(OCCN(CC)CC)C(I)=C2)C2...,Potassium voltage-gated channel subfamily H me...
...,...,...,...,...,...
5833,MVCOAUNKQVWQHZ-UHFFFAOYSA-N,MERRWPLGLGLVLLLCAPLPPGARAKEVTLMDTSKAQGELGWLLDP...,1-(5-Tert-Butyl-2-P-Tolyl-2h-Pyrazol-3-Yl)-3-[...,CC1=CC=C(C=C1)N1N=C(C=C1NC(=O)NC1=CC=C(OCCN2CC...,Ephrin type-A receptor 1
5834,CUIHSIWYWATEQL-UHFFFAOYSA-N,MGTSHPAFLVLGCLLTGLSLILCQLSLPSILPNENEKVVQLNSSFS...,Pazopanib,CN(C1=CC2=NN(C)C(C)=C2C=C1)C1=CC=NC(NC2=CC=C(C...,Platelet-derived growth factor receptor alpha
5835,JZUFKLXOESDKRF-UHFFFAOYSA-N,MASPDWGYDDKNGPEQWSKLYPIANGNNQSPVDIKTSETKHDTSLK...,Hydrochlorothiazide,NS(=O)(=O)C1=C(Cl)C=C2NCNS(=O)(=O)C2=C1,Carbonic anhydrase 1
5837,CEUORZQYGODEFX-UHFFFAOYSA-N,MGSLQPDAGNASWNGTEAPGGGARATPYSLQVTLTLVCLAGLLMLL...,Aripiprazole,ClC1=CC=CC(N2CCN(CCCCOC3=CC4=C(CCC(=O)N4)C=C3)...,error


In [57]:
drugbank_bd_df_binding['Y'] = 1
drugbank_bd_df_binding['is_ndm'] = 0
drugbank_bd_df_binding['is_bindingDB'] = 1

In [58]:
fulldb_smiles_binding_full = pd.concat([fulldb_smiles_binding,drugbank_bd_df_binding])

In [59]:
fulldb_smiles_binding_full

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB
30,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Prothrombin,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...,1,0,0
31,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Myeloperoxidase,MGVPFFSSLRCMVDLGPCWAGGLTAEMKLLLALAGLLAILATPQPS...,1,0,0
32,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Gonadotropin-releasing hormone receptor,MANSASPEQNQNHCSAINNSIPLMQGNLPTLTLSGKIRVTVTFFLF...,1,0,0
33,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Cytochrome P450 3A4,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,1,0,0
56,Goserelin,BLCLNMBMMGCOAS-URPVMXJPSA-N,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,Lutropin-choriogonadotropic hormone receptor,MKQRFSALQLLKLLLLLQPPLPRALREALCPEPCNCVPDGALRCPG...,1,0,0
...,...,...,...,...,...,...,...,...
5833,1-(5-Tert-Butyl-2-P-Tolyl-2h-Pyrazol-3-Yl)-3-[...,MVCOAUNKQVWQHZ-UHFFFAOYSA-N,CC1=CC=C(C=C1)N1N=C(C=C1NC(=O)NC1=CC=C(OCCN2CC...,Ephrin type-A receptor 1,MERRWPLGLGLVLLLCAPLPPGARAKEVTLMDTSKAQGELGWLLDP...,1,0,1
5834,Pazopanib,CUIHSIWYWATEQL-UHFFFAOYSA-N,CN(C1=CC2=NN(C)C(C)=C2C=C1)C1=CC=NC(NC2=CC=C(C...,Platelet-derived growth factor receptor alpha,MGTSHPAFLVLGCLLTGLSLILCQLSLPSILPNENEKVVQLNSSFS...,1,0,1
5835,Hydrochlorothiazide,JZUFKLXOESDKRF-UHFFFAOYSA-N,NS(=O)(=O)C1=C(Cl)C=C2NCNS(=O)(=O)C2=C1,Carbonic anhydrase 1,MASPDWGYDDKNGPEQWSKLYPIANGNNQSPVDIKTSETKHDTSLK...,1,0,1
5837,Aripiprazole,CEUORZQYGODEFX-UHFFFAOYSA-N,ClC1=CC=CC(N2CCN(CCCCOC3=CC4=C(CCC(=O)N4)C=C3)...,error,MGSLQPDAGNASWNGTEAPGGGARATPYSLQVTLTLVCLAGLLMLL...,1,0,1


In [60]:
fulldb_smiles_binding_full_dropped = fulldb_smiles_binding_full.drop_duplicates(subset=['InChiKey','aaseq'])

In [61]:
fulldb_smiles_binding_full_dropped

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB
30,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Prothrombin,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...,1,0,0
31,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Myeloperoxidase,MGVPFFSSLRCMVDLGPCWAGGLTAEMKLLLALAGLLAILATPQPS...,1,0,0
32,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Gonadotropin-releasing hormone receptor,MANSASPEQNQNHCSAINNSIPLMQGNLPTLTLSGKIRVTVTFFLF...,1,0,0
33,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Cytochrome P450 3A4,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,1,0,0
56,Goserelin,BLCLNMBMMGCOAS-URPVMXJPSA-N,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,Lutropin-choriogonadotropic hormone receptor,MKQRFSALQLLKLLLLLQPPLPRALREALCPEPCNCVPDGALRCPG...,1,0,0
...,...,...,...,...,...,...,...,...
5831,PD173955,VAARYSWULJUGST-UHFFFAOYSA-N,CSC1=CC=CC(NC2=NC=C3C=C(C(=O)N(C)C3=N2)C2=C(Cl...,Receptor-type tyrosine-protein kinase FLT3,MPALARDGGQLPLLVVFSAMIFGTITNQDLPVIKCVLINHKNNDSS...,1,0,1
5832,"(13R,15S)-13-METHYL-16-OXA-8,9,12,22,24-PENTAA...",KBLPHMRCKHFBJB-OLZOCXBDSA-N,[H][C@]12CN(CCN3NC4=C(C=CC=C4C3=O)C3=NC4=C(NC3...,RAC-alpha serine/threonine-protein kinase,MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVD...,1,0,1
5833,1-(5-Tert-Butyl-2-P-Tolyl-2h-Pyrazol-3-Yl)-3-[...,MVCOAUNKQVWQHZ-UHFFFAOYSA-N,CC1=CC=C(C=C1)N1N=C(C=C1NC(=O)NC1=CC=C(OCCN2CC...,Ephrin type-A receptor 1,MERRWPLGLGLVLLLCAPLPPGARAKEVTLMDTSKAQGELGWLLDP...,1,0,1
5835,Hydrochlorothiazide,JZUFKLXOESDKRF-UHFFFAOYSA-N,NS(=O)(=O)C1=C(Cl)C=C2NCNS(=O)(=O)C2=C1,Carbonic anhydrase 1,MASPDWGYDDKNGPEQWSKLYPIANGNNQSPVDIKTSETKHDTSLK...,1,0,1


In [62]:
fulldb_smiles_binding_full_dropped.to_pickle('DrugBank_Positive_Samples_Clean.pkl')

# DrugBank Negative Samples

#### From network

In [68]:
drugbank_sp_df = joblib.load('Negative_Samples_over_13_path_len.pkl')

In [69]:
drugbank_sp_df

,actions,drug_CID,drug_SID,drug_groups,drug_id,drug_inchi,drug_inchi_key,drug_iupac,drug_name,drug_smiles,drug_type,organism,target_aa_sequence,target_category,target_gene_sequence,target_id,target_name,target_uniprot_id,path,path_len
0,,445375,None,experimental,DB03645,"InChI=1S/C2H6NO5P/c4-2(3-5)1-9(6,7)8/h5H,1H2,(...",LDKRAXXVBWHMRH-UHFFFAOYSA-N,[(hydroxycarbamoyl)methyl]phosphonic acid,Phosphonoacetohydroxamic Acid,ONC(=O)CP(O)(O)=O,small molecule,T3D,MSSMILTQFGPFIESISGITDQSNDVFEDAAKAFSMFTRSDVYKAL...,target,,BE0004792,RNA-directed RNA polymerase lambda-3,P0CK31,"[DB03645, BE0004460, DB01819, BE0002562, DB034...",13
1,,445375,None,experimental,DB03645,"InChI=1S/C2H6NO5P/c4-2(3-5)1-9(6,7)8/h5H,1H2,(...",LDKRAXXVBWHMRH-UHFFFAOYSA-N,[(hydroxycarbamoyl)methyl]phosphonic acid,Phosphonoacetohydroxamic Acid,ONC(=O)CP(O)(O)=O,small molecule,Mycobacterium tuberculosis H37Rv,MSDFDTERVSRAVAAALVGPGGVALVVKVFAGLPGVIHTPARRGFF...,target,ATGTCTGATTTCGATACCGAGCGAGTGAGCCGCGCCGTCGCGGCCG...,BE0004223,Uncharacterized protein,O06796,"[DB03645, BE0004460, DB01819, BE0002562, DB034...",13
2,,445375,None,experimental,DB03645,"InChI=1S/C2H6NO5P/c4-2(3-5)1-9(6,7)8/h5H,1H2,(...",LDKRAXXVBWHMRH-UHFFFAOYSA-N,[(hydroxycarbamoyl)methyl]phosphonic acid,Phosphonoacetohydroxamic Acid,ONC(=O)CP(O)(O)=O,small molecule,Humans,MESPSAPPHRWCIPWQRLLLTASLLTFWNPPTTAKLTIESTPFNVA...,target,ATGGAGTCTCCCTCGGCCCCTCCCCACAGATGGTGCATCCCCTGGC...,BE0004224,Carcinoembryonic antigen-related cell adhesion...,P06731,"[DB03645, BE0004460, DB01819, BE0002562, DB034...",13
3,,445375,None,experimental,DB03645,"InChI=1S/C2H6NO5P/c4-2(3-5)1-9(6,7)8/h5H,1H2,(...",LDKRAXXVBWHMRH-UHFFFAOYSA-N,[(hydroxycarbamoyl)methyl]phosphonic acid,Phosphonoacetohydroxamic Acid,ONC(=O)CP(O)(O)=O,small molecule,Escherichia coli,MKKLAIMAAASMVFAVSSAHAGFTPSGTTGTTKLTVTEECQVRVGD...,target,ATGAGGGAGCGATATCTGTATCTTGCTGACACCCCTCAGGGGATAC...,BE0004225,Afimbrial adhesin AFA-III,Q57254,"[DB03645, BE0004460, DB01819, BE0002562, DB034...",13
4,,445375,None,experimental,DB03645,"InChI=1S/C2H6NO5P/c4-2(3-5)1-9(6,7)8/h5H,1H2,(...",LDKRAXXVBWHMRH-UHFFFAOYSA-N,[(hydroxycarbamoyl)methyl]phosphonic acid,Phosphonoacetohydroxamic Acid,ONC(=O)CP(O)(O)=O,small molecule,Mycobacterium tuberculosis,MSPAPVQVMGVLNVTDDSFSDGGCYLDLDDAVKHGLAMAAAGAGIV...,target,GTGAGTCCGGCGCCCGTGCAGGTGATGGGGGTTCTAAACGTCACGG...,BE0001974,Dihydropteroate synthase 1,P9WND1,"[DB03645, BE0004460, DB01819, BE0003655, DB001...",13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19455,,"[444764.0, 444764.0]","[104012053.0, 104012053.0]",experimental,DB02622,InChI=1S/C9H7NO5/c11-7(9(14)15)10-6-4-2-1-3-5(...,QBYNNSFEMMNINN-UHFFFAOYSA-N,2-(carboxyformamido)benzoic acid,2-(Oxalyl-Amino)-Benzoic Acid,OC(=O)C(=O)NC1=C(C=CC=C1)C(O)=O,small molecule,Escherichia coli,MKKLAIMAAASMVFAVSSAHAGFTPSGTTGTTKLTVTEECQVRVGD...,target,ATGAGGGAGCGATATCTGTATCTTGCTGACACCCCTCAGGGGATAC...,BE0004225,Afimbrial adhesin AFA-III,Q57254,"[DB02622, BE0000623, DB02615, BE0000123, DB002...",13
19456,,"[444764.0, 444764.0]","[104012053.0, 104012053.0]",experimental,DB02622,InChI=1S/C9H7NO5/c11-7(9(14)15)10-6-4-2-1-3-5(...,QBYNNSFEMMNINN-UHFFFAOYSA-N,2-(carboxyformamido)benzoic acid,2-(Oxalyl-Amino)-Benzoic Acid,OC(=O)C(=O)NC1=C(C=CC=C1)C(O)=O,small molecule,Mycobacterium tuberculosis,MSPAPVQVMGVLNVTDDSFSDGGCYLDLDDAVKHGLAMAAAGAGIV...,target,GTGAGTCCGGCGCCCGTGCAGGTGATGGGGGTTCTAAACGTCACGG...,BE0001974,Dihydropteroate synthase 1,P9WND1,"[DB02622, BE0000623, DB02615, BE0000123, DB002...",13
19457,,"[5494422.0, 5494422.0]","[46512091.0, 46512091.0]",experimental,DB07779,InChI=1S/C22H26FN3O2/c23-20-11-5-4-9-17(20)13-...,ANQHSFFUNMTTRS-MOPGFXCFSA-N,N-{[(2S)-1-[(3R)-3-amino-4-(2-fluorophenyl)but...,N-({(2S)-1-[(3R)-3-AMINO-4-(2-FLUOROPHENYL)BUT...,[H][C@](N)(CC(=O)N1CCC[C@@]1([H])CNC(=O)C1=CC=...,small molecule,Mycobacterium tuberculosis,MSPAPVQVMGVLNVTDDSFSDGGCYLDLDDAVKHGLAMAAAGAGIV...,target,GTGAGTCCGGCGCCCGTGCAGGTGATGGGGGTTCTAAACGTCACGG...,BE0001974,Dihydropteroate synt

In [70]:
drugbank_sp_df_smiles = drugbank_sp_df[['drug_name','drug_inchi_key','drug_smiles','target_name','target_aa_sequence']]
drugbank_sp_df_smiles = drugbank_sp_df_smiles.rename(columns={'drug_name':'chem_name','drug_inchi_key':'InChiKey','drug_smiles':'chem_smile','target_aa_sequence':'aaseq'})

In [71]:
drugbank_sp_df_smiles = drugbank_sp_df_smiles.drop_duplicates(subset=['InChiKey','aaseq'])
drugbank_sp_df_smiles['Y'] = 0
drugbank_sp_df_smiles['is_ndm'] = 0
drugbank_sp_df_smiles['is_bindingDB'] = 0

In [72]:
drugbank_sp_df_smiles

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB
0,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,RNA-directed RNA polymerase lambda-3,MSSMILTQFGPFIESISGITDQSNDVFEDAAKAFSMFTRSDVYKAL...,0,0,0
1,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,Uncharacterized protein,MSDFDTERVSRAVAAALVGPGGVALVVKVFAGLPGVIHTPARRGFF...,0,0,0
2,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,Carcinoembryonic antigen-related cell adhesion...,MESPSAPPHRWCIPWQRLLLTASLLTFWNPPTTAKLTIESTPFNVA...,0,0,0
3,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,Afimbrial adhesin AFA-III,MKKLAIMAAASMVFAVSSAHAGFTPSGTTGTTKLTVTEECQVRVGD...,0,0,0
4,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,Dihydropteroate synthase 1,MSPAPVQVMGVLNVTDDSFSDGGCYLDLDDAVKHGLAMAAAGAGIV...,0,0,0
...,...,...,...,...,...,...,...,...
19455,2-(Oxalyl-Amino)-Benzoic Acid,QBYNNSFEMMNINN-UHFFFAOYSA-N,OC(=O)C(=O)NC1=C(C=CC=C1)C(O)=O,Afimbrial adhesin AFA-III,MKKLAIMAAASMVFAVSSAHAGFTPSGTTGTTKLTVTEECQVRVGD...,0,0,0
19456,2-(Oxalyl-Amino)-Benzoic Acid,QBYNNSFEMMNINN-UHFFFAOYSA-N,OC(=O)C(=O)NC1=C(C=CC=C1)C(O)=O,Dihydropteroate synthase 1,MSPAPVQVMGVLNVTDDSFSDGGCYLDLDDAVKHGLAMAAAGAGIV...,0,0,0
19457,N-({(2S)-1-[(3R)-3-AMINO-4-(2-FLUOROPHENYL)BUT...,ANQHSFFUNMTTRS-MOPGFXCFSA-N,[H][C@](N)(CC(=O)N1CCC[C@@]1([H])CNC(=O)C1=CC=...,Dihydropteroate synthase 1,MSPAPVQVMGVLNVTDDSFSDGGCYLDLDDAVKHGLAMAAAGAGIV...,0,0,0
19458,"1,4-Dideoxy-5-Dehydro-O2-Sulfo-Glucuronic Acid",VJIMUKBSNUBECH-YKKSOZKNSA-N,[H][C@@]1(O)OC(=C[C@]([H])(O)[C@@]1([H])OS(O)(...,Dihydropteroate synthase 1,MSPAPVQVMGVLNVTDDSFSDGGCYLDLDDAVKHGLAMAAAGAGIV...,0,0,0


#### From BindingDB

In [36]:
db_pos_raw = joblib.load('bindingdictDBclean.pkl')

In [37]:
tr=10**6
labelinactive=[k for k,v in db_pos_raw.items() if (sum(np.array(list(v.values()))>=tr)>=len(v.values()))]
len(labelinactive)

395

In [38]:
drugbank_bd_df_neg = pd.DataFrame()

inchilist = []
aalist = []

for pair in labelinactive:
    inchilist.append(pair[0])
    aalist.append(pair[1])
    
drugbank_bd_df_neg['InChiKey'] = inchilist
drugbank_bd_df_neg['aaseq'] = aalist

In [39]:
to_drop_noDB = []
drop_chems = []

for index, row in drugbank_bd_df_neg.iterrows():
    try:
        drugbank_bd_df_neg.at[index,'chem_name'] = chem_dict[row['InChiKey']]['chem_name'] 
        drugbank_bd_df_neg.at[index,'chem_smile'] = chem_dict[row['InChiKey']]['chem_smile']
        drugbank_bd_df_neg.at[index,'target_name'] = target_dict[row['aaseq']]
    except:
        to_drop_noDB.append(index)
        drop_chems.append(row['InChiKey'])
        
drugbank_bd_df_non_binding = drugbank_bd_df_neg.drop(to_drop_noDB)

In [40]:
len(set(to_drop_noDB)), len(set(drop_chems))

(0, 0)

In [41]:
drugbank_bd_df_non_binding['Y'] = 0
drugbank_bd_df_non_binding['is_ndm'] = 0
drugbank_bd_df_non_binding['is_bindingDB'] = 1

In [42]:
drugbank_bd_df_non_binding = drugbank_bd_df_non_binding.drop_duplicates(subset=['InChiKey','aaseq'])

In [45]:
drugbank_bd_df_non_binding.to_csv('drugbank_bd_df_non_binding.csv')

In [44]:
drugbank_all_neg = pd.concat([drugbank_sp_df_smiles,drugbank_bd_df_non_binding])

NameError: name 'drugbank_sp_df_smiles' is not defined

In [82]:
drugbank_all_neg

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB
0,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,RNA-directed RNA polymerase lambda-3,MSSMILTQFGPFIESISGITDQSNDVFEDAAKAFSMFTRSDVYKAL...,0,0,0
1,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,Uncharacterized protein,MSDFDTERVSRAVAAALVGPGGVALVVKVFAGLPGVIHTPARRGFF...,0,0,0
2,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,Carcinoembryonic antigen-related cell adhesion...,MESPSAPPHRWCIPWQRLLLTASLLTFWNPPTTAKLTIESTPFNVA...,0,0,0
3,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,Afimbrial adhesin AFA-III,MKKLAIMAAASMVFAVSSAHAGFTPSGTTGTTKLTVTEECQVRVGD...,0,0,0
4,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,Dihydropteroate synthase 1,MSPAPVQVMGVLNVTDDSFSDGGCYLDLDDAVKHGLAMAAAGAGIV...,0,0,0
...,...,...,...,...,...,...,...,...
385,Diazepam,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C2=C(C=C(Cl)C=C2)C(=NCC1=O)C1=CC=CC=C1,Translocator protein,MAPPWVPAMGFTLAPSLGCFVGSRFVHGEGLRWYAGLQKPSWHPPH...,0,0,1
387,Acetohydroxamic acid,RRUDCFGSUDOHDG-UHFFFAOYSA-N,CC(=O)NO,72 kDa type IV collagenase,MEALMARGALTGPLRALCLLGCLLSHAAAAPSPIIKFPGDVAPKTD...,0,0,1
389,Methyldopa,CJCSPKMFHVPWAR-JTQLQIEISA-N,C[C@](N)(CC1=CC=C(O)C(O)=C1)C(O)=O,Cystathionine gamma-lyase,MQEKDASSQGFLPHFQHFATQAIHVGQDPEQWTSRAVVPPISLSTT...,0,0,1
391,S-METHYL-GLUTATHIONE,QTQDDTSVRVWHMO-BQBZGAKWSA-N,CSC[C@H](NC(=O)CC[C@H](N)C(O)=O)C(=O)NCC(O)=O,Lactoylglutathione lyase,MAEPQPPSGGLTDEAALSCCSDADPSTKDFLLQQTMLRVKDPKKSL...,0,0,1


In [83]:
drugbank_all_neg = drugbank_all_neg.drop_duplicates(subset=['InChiKey','aaseq'])
drugbank_all_neg

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB
0,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,RNA-directed RNA polymerase lambda-3,MSSMILTQFGPFIESISGITDQSNDVFEDAAKAFSMFTRSDVYKAL...,0,0,0
1,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,Uncharacterized protein,MSDFDTERVSRAVAAALVGPGGVALVVKVFAGLPGVIHTPARRGFF...,0,0,0
2,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,Carcinoembryonic antigen-related cell adhesion...,MESPSAPPHRWCIPWQRLLLTASLLTFWNPPTTAKLTIESTPFNVA...,0,0,0
3,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,Afimbrial adhesin AFA-III,MKKLAIMAAASMVFAVSSAHAGFTPSGTTGTTKLTVTEECQVRVGD...,0,0,0
4,Phosphonoacetohydroxamic Acid,LDKRAXXVBWHMRH-UHFFFAOYSA-N,ONC(=O)CP(O)(O)=O,Dihydropteroate synthase 1,MSPAPVQVMGVLNVTDDSFSDGGCYLDLDDAVKHGLAMAAAGAGIV...,0,0,0
...,...,...,...,...,...,...,...,...
385,Diazepam,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C2=C(C=C(Cl)C=C2)C(=NCC1=O)C1=CC=CC=C1,Translocator protein,MAPPWVPAMGFTLAPSLGCFVGSRFVHGEGLRWYAGLQKPSWHPPH...,0,0,1
387,Acetohydroxamic acid,RRUDCFGSUDOHDG-UHFFFAOYSA-N,CC(=O)NO,72 kDa type IV collagenase,MEALMARGALTGPLRALCLLGCLLSHAAAAPSPIIKFPGDVAPKTD...,0,0,1
389,Methyldopa,CJCSPKMFHVPWAR-JTQLQIEISA-N,C[C@](N)(CC1=CC=C(O)C(O)=C1)C(O)=O,Cystathionine gamma-lyase,MQEKDASSQGFLPHFQHFATQAIHVGQDPEQWTSRAVVPPISLSTT...,0,0,1
391,S-METHYL-GLUTATHIONE,QTQDDTSVRVWHMO-BQBZGAKWSA-N,CSC[C@H](NC(=O)CC[C@H](N)C(O)=O)C(=O)NCC(O)=O,Lactoylglutathione lyase,MAEPQPPSGGLTDEAALSCCSDADPSTKDFLLQQTMLRVKDPKKSL...,0,0,1


In [84]:
drugbank_all_neg.to_pickle('DrugBank_Negative_Samples_Clean.pkl')

# NDM Positive Samples

#### From BindingDB

In [61]:
ndm_pos_bd = joblib.load('bindingdictNDMclean.pkl')

In [62]:
ndm_bd_pairs = list(set(ndm_pos_bd.keys()))

In [63]:
tr=10**3
labelactive=[k for k,v in ndm_pos_bd.items() if (sum(np.array(list(v.values()))<=tr)>0)]
len(labelactive)

1262

In [64]:
ndm_bd_df = pd.DataFrame()

inchilist = []
aalist = []

for pair in labelactive:
    inchilist.append(pair[0])
    aalist.append(pair[1])
    
ndm_bd_df['InChiKey'] = inchilist
ndm_bd_df['aaseq'] = aalist

In [65]:
ndm = pd.read_csv('NDM_Master.csv')

inchikey_chemname_dict = dict()

for index, row in ndm.iterrows():
    inchikey_chemname_dict[row['InChIKey']] = dict()
    inchikey_chemname_dict[row['InChIKey']]['chem_name'] = row['name']
    inchikey_chemname_dict[row['InChIKey']]['chem_smile'] = row['SMILE']
    
# target_dict for BindingDB targets

In [66]:
ndm_genes_deisy = pd.read_csv('NDM Genes/missingGenes_Gene_Sequence.csv')

In [67]:
for index, row in ndm_genes_deisy.iterrows():
    target_dict[row['Sequence']] = row['gene']

In [68]:
for index ,row in ndm_bd_df.iterrows():
        ndm_bd_df.at[index,'chem_name'] = inchikey_chemname_dict[row['InChiKey']]['chem_name'] 
        ndm_bd_df.at[index,'chem_smile'] = inchikey_chemname_dict[row['InChiKey']]['chem_smile']
        ndm_bd_df.at[index,'target_name'] = target_dict[row['aaseq']]
        
ndm_bd_df_binding = ndm_bd_df

In [69]:
ndm_bd_df_binding['Y'] = 1
ndm_bd_df_binding['is_ndm'] = 1
ndm_bd_df_binding['is_bindingDB'] = 1

In [70]:
ndm_bd_df_binding

,InChiKey,aaseq,chem_name,chem_smile,target_name,Y,is_ndm,is_bindingDB
0,UREBDLICKHMUKA-CXSFZGCWSA-N,MKGLLPLAWFLACSVPAVQGGLLDLKSMIEKVTGKNALTNYGFYGC...,Dexamethasone,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@@]4([C@...,"Secretory phospholipase A2, group V (sPLA2-V)",1,1,1
1,BXNJHAXVSOCGBA-UHFFFAOYSA-N,MHTGGETSACKPSSVRLAPSFSFHAAGLQMAGQMPHSHQYSDRRQP...,Harmine,CC1=NC=CC2=C1NC3=C2C=CC(=C3)OC,DYRK1A,1,1,1
2,QAIPRVGONGVQAS-DUXPYHPUSA-N,MEALMARGALTGPLRALCLLGCLLSHAAAAPSPIIKFPGDVAPKTD...,trans-Caffeic acid,C1=CC(=C(C=C1/C=C/C(=O)O)O)O,72 kDa type IV collagenase,1,1,1
3,QXKHYNVANLEOEG-UHFFFAOYSA-N,MLASGLLLVTLLACLTVMVLMSVWRQRKSRGKLPPGPTPLPFIGNY...,Xanthotoxin,COC1=C2C(=CC3=C1OC=C3)C=CC(=O)O2,CYP2A13,1,1,1
4,SHGAZHPCJJPHSC-YCNIQYBTSA-N,MPNFAGTWKMRSSENFDELLKALGVNAMLRKVAVAAASKPHVEIRQ...,Retinoic acid,CC1=C(C(CCC1)(C)C)/C=C/C(=C/C=C/C(=C/C(=O)O)/C)/C,Cellular retinoic acid-binding protein I,1,1,1
...,...,...,...,...,...,...,...,...
1257,ISIMGBQRFXXNON-FCXRPNKRSA-N,MAFMKKYLLPILGLFMAYYYYSANEEFRPEMLQGKKVIVTGASKGI...,"1,5-bis-(4-Hydroxy-3-methoxyphenyl)-penta-tran...",COC1=C(C=CC(=C1)/C=C/C(=O)/C=C/C2=CC(=C(C=C2)O...,HSD11B1,1,1,1
1258,WMBWREPUVVBILR-NQIIRXRSSA-N,MNKISQRLLFLFLHFYTTVCFIQNNTQKTFHNVLQNEQIRGKEKAF...,ent-Gallocatechin 3-gallate,C1[C@H]([C@@H](OC2=CC(=CC(=C21)O)O)C3=CC(=C(C(...,Enoyl-acyl-carrier protein reductase,1,1,1
1259,APJYDQYYACXCRM-UHFFFAOYSA-N,MSPLNQSAEGLPQEASNRSLNATETSEAWDPRTLQALKISLAVVLS...,Tryptamine,C1=CC=C2C(=C1)C(=CN2)CCN,HTR1D,1,1,1
1260,PXGPLTODNUVGFL-RTYMFESYSA-N,MADAQNISLDSPGSVGAVAVPVVFALIFLLGTVGNGLVLAVLLQPG...,Prostaglandin F-2-alpha,CCCCCC(/C=C/C1C(CC(C1C/C=C/CCCC(=O)O)O)O)O,Galanin receptor 3,1,1,1


In [71]:
bioseq_uniprot_dict = dict()

for index, row in ndm_bd_df_binding.iterrows():
    bioseq_uniprot_dict[row['aaseq']] = row['aaseq']

#### NDM from DTC

In [72]:
ndm_pos_dtc = joblib.load('NDMdbDTC4network.pkl')

In [73]:
ndm_pos_dtc

,compound_id,standard_inchi_key,compound_name,synonym,target_id,target_pref_name,gene_names,wildtype_or_mutant,mutation_info,pubmed_id,...,substrate_units,assay_description,title,journal,doc_type,annotation_comments,gene_seq,InChi,is_NDM,component
6836,CHEMBL12,AAOVKJBEBIDNHE-UHFFFAOYSA-N,DIAZEPAM,NaN,P47869,GABA RECEPTOR ALPHA-2 SUBUNIT,GABRA2,NaN,NaN,2551039.0,...,NaN,NaN,NaN,NaN,NaN,NaN,ATGCTCCTTGTACTTCAGAACGTCTCGCGCCACCAGGACCGCTTCT...,AAOVKJBEBIDNHE-UHFFFAOYSA-N,1,1
6867,CHEMBL12,AAOVKJBEBIDNHE-UHFFFAOYSA-N,DIAZEPAM,NaN,P34903,GABA RECEPTOR ALPHA-3 SUBUNIT,GABRA3,NaN,NaN,2551039.0,...,NaN,NaN,NaN,NaN,NaN,NaN,ATGGCCACCTTTACCTTCCGGGAAGCCTGGCGCAAGAAGGTCGTCC...,AAOVKJBEBIDNHE-UHFFFAOYSA-N,1,1
6882,CHEMBL12,AAOVKJBEBIDNHE-UHFFFAOYSA-N,DIAZEPAM,NaN,P31644,GABA RECEPTOR ALPHA-5 SUBUNIT,GABRA5,NaN,NaN,18537233.0,...,NaN,Binding affinity to GABAA alpha-5-beta-2-gamma...,Selective influence on contextual memory: phys...,J. Med. Chem.,PUBLICATION,NaN,ATGATCCGGCAGCATAAATGGTTTTCCGGCGTAGCCGCGGCCGTCC...,AAOVKJBEBIDNHE-UHFFFAOYSA-N,1,1
7155,CHEMBL12,AAOVKJBEBIDNHE-UHFFFAOYSA-N,DIAZEPAM,NaN,P14867,GABA RECEPTOR ALPHA-1 SUBUNIT,GABRA1,NaN,NaN,2551039.0,...,NaN,NaN,NaN,NaN,NaN,NaN,ATGAGGGTCTTAGTTGATTCCAGTGCCATAGCGGCCTTAATCCTGA...,AAOVKJBEBIDNHE-UHFFFAOYSA-N,1,1
7156,CHEMBL12,AAOVKJBEBIDNHE-UHFFFAOYSA-N,DIAZEPAM,NaN,P14867,GABA RECEPTOR ALPHA-1 SUBUNIT,GABRA1,NaN,NaN,8632757.0,...,NaN,NaN,NaN,NaN,NaN,NaN,ATGAGGGTCTTAGTTGATTCCAGTGCCATAGCGGCCTTAATCCTGA...,AAOVKJBEBIDNHE-UHFFFAOYSA-N,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5886186,CHEMBL35,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,FUROSEMIDE,NaN,P22748,CARBONIC ANHYDRASE IV,CA4,NaN,NaN,19119014.0,...,NaN,Inhibition of human recombinant full length CA...,Carbonic anhydrase inhibitors. Comparison of c...,Bioorg. Med. Chem.,PUBLICATION,NaN,ATGGCCTTGCGCTCTTTACGCTGGAAGATCGCCTTAAACTTTTTAA...,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,1,1
5886309,CHEMBL35,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,FUROSEMIDE,NaN,P00918,CARBONIC ANHYDRASE II,CA2,NaN,NaN,19119014.0,...,NaN,Inhibition of human recombinant full length CA...,Carbonic anhydrase inhibitors. Comparison of c...,Bioorg. Med. Chem.,PUBLICATION,NaN,GTGATCTTCCTGGCGAAAACAAAAGAGCGCTTTGTCTGCCAGCAAT...,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,1,1
5886314,CHEMBL35,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,FUROSEMIDE,NaN,P00915,CARBONIC ANHYDRASE I,CA1,NaN,NaN,19119014.0,...,NaN,Inhibition of human recombinant full length CA...,Carbonic anhydrase inhibitors. Comparison of c...,Bioorg. Med. Chem.,PUBLICATION,NaN,ATGTTTAAAGTGGGCGATAAAGTAGTGTATCCCATGCACGGGGCGG...,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,1,1
5886334,CHEMBL35,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,FUROSEMIDE,NaN,O43570,CARBONIC ANHYDRASE XII,CA12,NaN,NaN,19119014.0,...,NaN,Inhibition of human recombinant CA12 catalytic...,Carbonic anhydrase inhibitors. Comparison of c...,Bioorg. Med. Chem.,PUBLICATION,NaN,ATGAACAGGCTGGATGGTTTTAACATGCCAGAGCCCAACTGGGCCC...,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,1,1


In [74]:
dtc_genes_uniprot = pd.read_csv('NDM Genes/missingGenes_Gene_Sequence.csv')

dtc_genes_uniprot_dict = dict()

for index, row in dtc_genes_uniprot.iterrows():
    dtc_genes_uniprot_dict[row['gene']] = row['Sequence']

In [75]:
## Mapping with BioSeq 

dtc_genes_bioseq = pd.read_csv('../Deisy/NDM_Target_Info.csv')

for index, row in dtc_genes_bioseq.iterrows():
    bioseq_uniprot_dict[row['aaseq_BioSeq']] = dtc_genes_uniprot_dict[row['gene_names']]

In [76]:
for index, row in ndm_pos_dtc.iterrows():
    ndm_pos_dtc.at[index,'aaseq'] = dtc_genes_uniprot_dict[row['gene_names']] 

In [77]:
for index ,row in ndm_pos_dtc.iterrows():
        ndm_pos_dtc.at[index,'chem_name'] = inchikey_chemname_dict[row['InChi']]['chem_name'] 
        ndm_pos_dtc.at[index,'chem_smile'] = inchikey_chemname_dict[row['InChi']]['chem_smile']

In [78]:
ndm_pos_dtc_smiles = ndm_pos_dtc[['compound_name','InChi','chem_smile','target_pref_name','aaseq']]
ndm_pos_dtc_smiles = ndm_pos_dtc_smiles.rename(columns={'compound_name':'chem_name','InChi':'InChiKey','target_pref_name':'target_name'})

In [79]:
ndm_pos_dtc_smiles

,chem_name,InChiKey,chem_smile,target_name,aaseq
6836,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,GABA RECEPTOR ALPHA-2 SUBUNIT,MKTKLNIYNMQFLLFVFLVWDPARLVLANIQEDEAKNNITIFTRIL...
6867,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,GABA RECEPTOR ALPHA-3 SUBUNIT,MIITQTSHCYMTSLGILFLINILPGTTGQGESRRQEPGDFVKQDIG...
6882,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,GABA RECEPTOR ALPHA-5 SUBUNIT,MDNGMFSGFIMIKNLLLFCISMNLSSHFGFSQMPTSSVKDETNDNI...
7155,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,GABA RECEPTOR ALPHA-1 SUBUNIT,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...
7156,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,GABA RECEPTOR ALPHA-1 SUBUNIT,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...
...,...,...,...,...,...
5886186,FUROSEMIDE,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl,CARBONIC ANHYDRASE IV,MRMLLALLALSAARPSASAESHWCYEVQAESSNYPCLVPVKWGGNC...
5886309,FUROSEMIDE,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl,CARBONIC ANHYDRASE II,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...
5886314,FUROSEMIDE,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl,CARBONIC ANHYDRASE I,MASPDWGYDDKNGPEQWSKLYPIANGNNQSPVDIKTSETKHDTSLK...
5886334,FUROSEMIDE,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl,CARBONIC ANHYDRASE XII,MPRRSLHAAAVLLLVILKEQPSSPAPVNGSKWTYFGPDGENSWSKK...


In [80]:
ndm_pos_dtc_smiles = ndm_pos_dtc_smiles.drop_duplicates(subset=['InChiKey','aaseq'])
ndm_pos_dtc_smiles['Y'] = 1
ndm_pos_dtc_smiles['is_ndm'] = 1
ndm_pos_dtc_smiles['is_bindingDB'] = 0
ndm_pos_dtc_smiles

C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB
6836,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,GABA RECEPTOR ALPHA-2 SUBUNIT,MKTKLNIYNMQFLLFVFLVWDPARLVLANIQEDEAKNNITIFTRIL...,1,1,0
6867,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,GABA RECEPTOR ALPHA-3 SUBUNIT,MIITQTSHCYMTSLGILFLINILPGTTGQGESRRQEPGDFVKQDIG...,1,1,0
6882,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,GABA RECEPTOR ALPHA-5 SUBUNIT,MDNGMFSGFIMIKNLLLFCISMNLSSHFGFSQMPTSSVKDETNDNI...,1,1,0
7155,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,GABA RECEPTOR ALPHA-1 SUBUNIT,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,1,1,0
7179,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,ADENOSINE A3 RECEPTOR,MPNNSTALSLANVTYITMEIFIGLCAIVGNVLVICVVKLNPSLQTT...,1,1,0
...,...,...,...,...,...,...,...,...
5886186,FUROSEMIDE,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl,CARBONIC ANHYDRASE IV,MRMLLALLALSAARPSASAESHWCYEVQAESSNYPCLVPVKWGGNC...,1,1,0
5886309,FUROSEMIDE,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl,CARBONIC ANHYDRASE II,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,1,1,0
5886314,FUROSEMIDE,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl,CARBONIC ANHYDRASE I,MASPDWGYDDKNGPEQWSKLYPIANGNNQSPVDIKTSETKHDTSLK...,1,1,0
5886334,FUROSEMIDE,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl,CARBONIC ANHYDRASE XII,MPRRSLHAAAVLLLVILKEQPSSPAPVNGSKWTYFGPDGENSWSKK...,1,1,0


In [81]:
ndm_all_pos = pd.concat([ndm_bd_df_binding,ndm_pos_dtc_smiles])

In [82]:
ndm_all_pos = ndm_all_pos.drop_duplicates(subset=['InChiKey','aaseq'])

In [83]:
ndm_all_pos

,InChiKey,aaseq,chem_name,chem_smile,target_name,Y,is_ndm,is_bindingDB
0,UREBDLICKHMUKA-CXSFZGCWSA-N,MKGLLPLAWFLACSVPAVQGGLLDLKSMIEKVTGKNALTNYGFYGC...,Dexamethasone,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@@]4([C@...,"Secretory phospholipase A2, group V (sPLA2-V)",1,1,1
1,BXNJHAXVSOCGBA-UHFFFAOYSA-N,MHTGGETSACKPSSVRLAPSFSFHAAGLQMAGQMPHSHQYSDRRQP...,Harmine,CC1=NC=CC2=C1NC3=C2C=CC(=C3)OC,DYRK1A,1,1,1
2,QAIPRVGONGVQAS-DUXPYHPUSA-N,MEALMARGALTGPLRALCLLGCLLSHAAAAPSPIIKFPGDVAPKTD...,trans-Caffeic acid,C1=CC(=C(C=C1/C=C/C(=O)O)O)O,72 kDa type IV collagenase,1,1,1
3,QXKHYNVANLEOEG-UHFFFAOYSA-N,MLASGLLLVTLLACLTVMVLMSVWRQRKSRGKLPPGPTPLPFIGNY...,Xanthotoxin,COC1=C2C(=CC3=C1OC=C3)C=CC(=O)O2,CYP2A13,1,1,1
4,SHGAZHPCJJPHSC-YCNIQYBTSA-N,MPNFAGTWKMRSSENFDELLKALGVNAMLRKVAVAAASKPHVEIRQ...,Retinoic acid,CC1=C(C(CCC1)(C)C)/C=C/C(=C/C=C/C(=C/C(=O)O)/C)/C,Cellular retinoic acid-binding protein I,1,1,1
...,...,...,...,...,...,...,...,...
5695088,ZESRJSPZRDMNHY-YFWFAHHUSA-N,MPLLLYTCLLWLPTSGLWTVQAMDPNAAYVNMSNHHRGLASANVDF...,DESOXYCORTICOSTERONE,C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@@H]2C(=O)CO)C...,CORTICOSTEROID BINDING GLOBULIN,1,1,0
5803024,ZQPPMHVWECSIRJ-KTKRTIGZSA-N,MAFDSTWKVDRSENYDKFMEKMGVNIVKRKLAAHDNLKLTITQEGN...,OLEIC ACID,CCCCCCCC/C=C\CCCCCCCC(=O)O,FATTY ACID BINDING PROTEIN INTESTINAL,1,1,0
5803034,ZQPPMHVWECSIRJ-KTKRTIGZSA-N,MSFSGKYQLQSQENFEAFMKAIGLPEELIQKGKDIKGVSEIVQNGK...,OLEIC ACID,CCCCCCCC/C=C\CCCCCCCC(=O)O,"FATTY ACID-BINDING PROTEIN, LIVER",1,1,0
5854123,ZWIADYZPOWUWEW-XVFCMESISA-N,MPNNSTALSLANVTYITMEIFIGLCAIVGNVLVICVVKLNPSLQTT...,NaN,C1=CN(C(=O)N=C1N)[C@H]2[C@@H]([C@@H]([C@H](O2)...,ADENOSINE A3 RECEPTOR,1,1,0


In [84]:
## Including DTC genes to target_dict 

for index, row in ndm_all_pos.iterrows():
    target_dict[row['aaseq']] = row['target_name']

In [179]:
ndm_all_pos.to_pickle('NDM_Positive_Samples_Clean.pkl')

# NDM Negative Samples

#### From DTC nonactive

In [85]:
nonactive = joblib.load('NDMnonactiveinDTC.pkl')

In [86]:
for index, row in nonactive.iterrows():
    nonactive.at[index,'aaseq'] = dtc_genes_uniprot_dict[row['gene_names']] 

In [87]:
for index ,row in nonactive.iterrows():
        nonactive.at[index,'chem_name'] = inchikey_chemname_dict[row['standard_inchi_key']]['chem_name'] 
        nonactive.at[index,'chem_smile'] = inchikey_chemname_dict[row['standard_inchi_key']]['chem_smile']

In [88]:
nonactive_smiles = nonactive[['compound_name','standard_inchi_key','chem_smile','target_pref_name','aaseq']]
nonactive_smiles = nonactive_smiles.rename(columns={'compound_name':'chem_name','standard_inchi_key':'InChiKey','target_pref_name':'target_name'})

In [108]:
nonactive_smiles['Y'] = 0
nonactive_smiles['is_ndm'] = 1
nonactive_smiles['is_bindingDB'] = 2 # For DTC

In [109]:
nonactive_smiles

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB
6531,NaN,AAOFJKLTRKOQTQ-UHFFFAOYSA-N,CC1=C(C(=C(C=C1OC)OC)C(=O)C)O,CYTOCHROME P450 3A4,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,0,1,2
6760,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,ESTROGEN RECEPTOR BETA,MDIKNSPSSLNSPSSYNCSQSILPLEHGSIYIPSSYVDSHHEYPAM...,0,1,2
6761,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,ESTROGEN RECEPTOR BETA,MDIKNSPSSLNSPSSYNCSQSILPLEHGSIYIPSSYVDSHHEYPAM...,0,1,2
6766,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,MAP KINASE P38 ALPHA,MSQERPTFYRQELNKTIWEVPERYQNLSPVGSGAYGSVCAAFDTKT...,0,1,2
6767,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,MAP KINASE P38 ALPHA,MSQERPTFYRQELNKTIWEVPERYQNLSPVGSGAYGSVCAAFDTKT...,0,1,2
...,...,...,...,...,...,...,...,...
5886311,FUROSEMIDE,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl,CARBONIC ANHYDRASE II,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0,1,2
5886312,FUROSEMIDE,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl,CARBONIC ANHYDRASE II,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0,1,2
5886317,FUROSEMIDE,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl,EPIDERMAL GROWTH FACTOR RECEPTOR ERBB1,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,0,1,2
5886318,FUROSEMIDE,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl,EPIDERMAL GROWTH FACTOR RECEPTOR ERBB1,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,0,1,2


#### From BindingDB

In [91]:
nonactive_bd = joblib.load('bindingdictNDMclean.pkl')

In [92]:
tr=10**6
labelinactive=[k for k,v in nonactive_bd.items() if (sum(np.array(list(v.values()))>=tr)>=len(v.values()))]
len(labelinactive)

486

In [93]:
ndm_bd_df_neg = pd.DataFrame()

inchilist = []
aalist = []

for pair in labelinactive:
    inchilist.append(pair[0])
    aalist.append(pair[1])
    
ndm_bd_df_neg['InChiKey'] = inchilist
ndm_bd_df_neg['aaseq'] = aalist

In [94]:
for index ,row in ndm_bd_df_neg.iterrows():
        ndm_bd_df_neg.at[index,'chem_name'] = inchikey_chemname_dict[row['InChiKey']]['chem_name'] 
        ndm_bd_df_neg.at[index,'chem_smile'] = inchikey_chemname_dict[row['InChiKey']]['chem_smile']
        ndm_bd_df_neg.at[index,'target_name'] = target_dict[row['aaseq']]

In [95]:
ndm_bd_df_neg['Y'] = 0
ndm_bd_df_neg['is_ndm'] = 1
ndm_bd_df_neg['is_bindingDB'] = 1

In [96]:
ndm_bd_df_neg

,InChiKey,aaseq,chem_name,chem_smile,target_name,Y,is_ndm,is_bindingDB
0,VZCYOOQTPOCHFL-OWOJBTEDSA-N,MAFTDLLDALGGVGRFQLVYTALLLLPCGLLACHTFLQNFTAAAPP...,Fumaric acid,C(=C/C(=O)O)\C(=O)O,Solute carrier family 22 member 20,0,1,1
1,UMGDCJDMYOKAJW-UHFFFAOYSA-N,MSTAGKVIKCKAAVLWEEKKPFSIEEVEVAPPKAHEVRIKMVATGI...,Thiourea,C(=N)(N)S,Alcohol dehydrogenase,0,1,1
2,VEXZGXHMUGYJMC-UHFFFAOYSA-M,MRFVSMIIKDILRENQDFRFRDLSDLKHSPKLCIITCMDSRLIDLL...,Chloride,[Cl-],Carbonic anhydrase,0,1,1
3,FXNFHKRTJBSTCS-UHFFFAOYSA-N,MASMLLAQRLACSFQHSYRLLVPGSRHISQAAAKVDVEFDYDGPLM...,Baicalein,C1=CC=C(C=C1)C2=CC(=O)C3=C(O2)C=C(C(=C3O)O)O,ABAT,0,1,1
4,HHVIBTZHLRERCL-UHFFFAOYSA-N,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,Dimethyl sulfone,CS(=O)(=O)C,Protein cereblon,0,1,1
...,...,...,...,...,...,...,...,...
481,FIDUIAPDSKSUGO-UHFFFAOYSA-N,MRPPQCLLHTPSLASPLLLLLLWLLGGGVGAEGREDAELLVTVRGG...,demethylsuberosin,CC(=CCC1=C(C=C2C(=C1)C=CC(=O)O2)O)C,ACETYLCHOLINESTERASE,0,1,1
482,RBIGKSZIQCTIJF-UHFFFAOYSA-N,MPSRTDPKMDRSGGRVRLKAHYGGDILITSVDPTTTFQDLCEEVRD...,Thiophen-3-carboxyaldehyde,C1=CSC=C1C=O,Protein kinase C (PKC),0,1,1
483,XMBWDFGMSWQBCA-UHFFFAOYSA-M,MRFVSMIIKDILRENQDFRFRDLSDLKHSPKLCIITCMDSRLIDLL...,Iodide,[I-],Carbonic anhydrase,0,1,1
484,QTQDDTSVRVWHMO-BQBZGAKWSA-N,MAEPQPPSGGLTDEAALSCCSDADPSTKDFLLQQTMLRVKDPKKSL...,S-methylglutathione,CSC[C@@H](C(=O)NCC(=O)O)NC(=O)CC[C@@H](C(=O)O)N,GLO1,0,1,1


#### From network

In [97]:
ndm_shortest_path = joblib.load('shortest_path_bp_NDM.pkl')

In [98]:
chem_list = []
gene_list = []
path_list = []
path_len_list =[]

for chem in ndm_shortest_path.keys():
    if '-' in str(chem):
        for gene in ndm_shortest_path[chem].keys():
            if '-' not in str(gene):
                chem_list.append(chem)
                gene_list.append(gene)
                path_list.append(ndm_shortest_path[chem][gene])
                path_len_list.append(len(ndm_shortest_path[chem][gene])-1)

In [99]:
shortest_path_df = pd.DataFrame()

shortest_path_df['InChiKey'] = chem_list
shortest_path_df['aaseq'] = gene_list
shortest_path_df['shortest_path'] = path_list
shortest_path_df['path_len'] = path_len_list

In [100]:
shortest_path_df

,InChiKey,aaseq,shortest_path,path_len
0,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,MAPLCPSPWLPLLIPAPAPGLTVQLLLSLLLLVPVHPQRLPRMQED...,"[ZZUFCTLCJUWOSV-UHFFFAOYSA-N, MAPLCPSPWLPLLIPA...",1
1,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,MARLSWGYGEHNGPIHWNELFPIADGDQQSPIEIKTKEVKYDSSLR...,"[ZZUFCTLCJUWOSV-UHFFFAOYSA-N, MARLSWGYGEHNGPIH...",1
2,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,MTGHHGWGYGQDDGPSHWHKLYPIAQGDRQSPINIISSQAVYSPSL...,"[ZZUFCTLCJUWOSV-UHFFFAOYSA-N, MTGHHGWGYGQDDGPS...",1
3,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,MLFSALLLEVIWILAADGGQHWTYEGPHGQDHWPASYPECGNNAQS...,"[ZZUFCTLCJUWOSV-UHFFFAOYSA-N, MLFSALLLEVIWILAA...",1
4,ZZUFCTLCJUWOSV-UHFFFAOYSA-N,MRMLLALLALSAARPSASAESHWCYEVQAESSNYPCLVPVKWGGNC...,"[ZZUFCTLCJUWOSV-UHFFFAOYSA-N, MRMLLALLALSAARPS...",1
...,...,...,...,...
195835,UXYRZJKIQKRJCF-TZPFWLJSSA-N,MSNRKKVAYFHDPDIGSYYYGAGHPMKPQRIRMTHSLIVSYNLYKY...,"[UXYRZJKIQKRJCF-TZPFWLJSSA-N, MESRGPLATSRLLLLL...",13
195836,UXYRZJKIQKRJCF-TZPFWLJSSA-N,MTSTGQDSSTRQRKSRHNPQSPLQESSATLKRGGKKCAVPHSSPNL...,"[UXYRZJKIQKRJCF-TZPFWLJSSA-N, MESRGPLATSRLLLLL...",13
195837,UXYRZJKIQKRJCF-TZPFWLJSSA-N,MAIGYVWNTLYGWVDTGTGSLAAANLTARMQPISHHLAHPDTKRRF...,"[UXYRZJKIQKRJCF-TZPFWLJSSA-N, MESRGPLATSRLLLLL...",13
195838,UXYRZJKIQKRJCF-TZPFWLJSSA-N,SSPITGLVYDQRMMLHHNMWDSHHPELPQRISRIFSRHEELRLLSR...,"[UXYRZJKIQKRJCF-TZPFWLJSSA-N, MESRGPLATSRLLLLL...",13


In [101]:
len(set(shortest_path_df['aaseq'].tolist()))

510

In [102]:
shortest_path_df_13 = shortest_path_df[shortest_path_df['path_len'] >= 13]
shortest_path_df_13 = shortest_path_df_13.drop_duplicates(subset=['InChiKey','aaseq'])
shortest_path_df_13 = shortest_path_df_13.drop(columns=['shortest_path','path_len'])
shortest_path_df_13

,InChiKey,aaseq
3540,WGNIVAMNAWBYRO-UHFFFAOYSA-N,MGSGPLSLPLALSPPRLLLLLLLSLLPVARASEAEHRLFERLFEDY...
3541,WGNIVAMNAWBYRO-UHFFFAOYSA-N,MRCSPGGVWLALAASLLHVSLQGEFQRKLYKELVKNYNPLERPVAN...
3542,WGNIVAMNAWBYRO-UHFFFAOYSA-N,MDQHQHLNKTAESASSEKKKTRRCNGFKMFLAALSFSYIAKALGGI...
3543,WGNIVAMNAWBYRO-UHFFFAOYSA-N,MASLKDLEGKWRLMESHGFEEYMKELGVGLALRKMAAMAKPDCIIT...
3544,WGNIVAMNAWBYRO-UHFFFAOYSA-N,MDQNQHLNKTAEAQPSENKKTRYCNGLKMFLAALSLSFIAKTLGAI...
...,...,...
195835,UXYRZJKIQKRJCF-TZPFWLJSSA-N,MSNRKKVAYFHDPDIGSYYYGAGHPMKPQRIRMTHSLIVSYNLYKY...
195836,UXYRZJKIQKRJCF-TZPFWLJSSA-N,MTSTGQDSSTRQRKSRHNPQSPLQESSATLKRGGKKCAVPHSSPNL...
195837,UXYRZJKIQKRJCF-TZPFWLJSSA-N,MAIGYVWNTLYGWVDTGTGSLAAANLTARMQPISHHLAHPDTKRRF...
195838,UXYRZJKIQKRJCF-TZPFWLJSSA-N,SSPITGLVYDQRMMLHHNMWDSHHPELPQRISRIFSRHEELRLLSR...


In [103]:
## Mapping the amino acid sequences

unavailable_seq = []

for index ,row in shortest_path_df_13.iterrows():
    try:
        shortest_path_df_13.at[index,'aaseq'] = bioseq_uniprot_dict[row['aaseq']]
    except:
        unavailable_seq.append(row['aaseq'])

In [104]:
len(set(unavailable_seq))

30

In [105]:
for index ,row in shortest_path_df_13.iterrows():
        shortest_path_df_13.at[index,'chem_name'] = inchikey_chemname_dict[row['InChiKey']]['chem_name'] 
        shortest_path_df_13.at[index,'chem_smile'] = inchikey_chemname_dict[row['InChiKey']]['chem_smile']
        shortest_path_df_13.at[index,'target_name'] = target_dict[row['aaseq']]

In [106]:
shortest_path_df_13_non_binding = shortest_path_df_13.drop_duplicates(subset=['InChiKey','aaseq'])
shortest_path_df_13_non_binding['Y'] = 0
shortest_path_df_13_non_binding['is_ndm'] = 1
shortest_path_df_13_non_binding['is_bindingDB'] = 0

In [107]:
shortest_path_df_13_non_binding

,InChiKey,aaseq,chem_name,chem_smile,target_name,Y,is_ndm,is_bindingDB
3540,WGNIVAMNAWBYRO-UHFFFAOYSA-N,MGSGPLSLPLALSPPRLLLLLLLSLLPVARASEAEHRLFERLFEDY...,Gancaonin Q,CC(=CCC1=C(C=CC(=C1)C2=CC(=O)C3=C(O2)C=C(C(=C3...,NEURONAL ACETYLCHOLINE RECEPTOR SUBUNIT ALPHA-3,0,1,0
3541,WGNIVAMNAWBYRO-UHFFFAOYSA-N,MRCSPGGVWLALAASLLHVSLQGEFQRKLYKELVKNYNPLERPVAN...,Gancaonin Q,CC(=CCC1=C(C=CC(=C1)C2=CC(=O)C3=C(O2)C=C(C(=C3...,NEURONAL ACETYLCHOLINE RECEPTOR PROTEIN ALPHA-...,0,1,0
3542,WGNIVAMNAWBYRO-UHFFFAOYSA-N,MDQHQHLNKTAESASSEKKKTRRCNGFKMFLAALSFSYIAKALGGI...,Gancaonin Q,CC(=CCC1=C(C=CC(=C1)C2=CC(=O)C3=C(O2)C=C(C(=C3...,Solute carrier organic anion transporter famil...,0,1,0
3543,WGNIVAMNAWBYRO-UHFFFAOYSA-N,MASLKDLEGKWRLMESHGFEEYMKELGVGLALRKMAAMAKPDCIIT...,Gancaonin Q,CC(=CCC1=C(C=CC(=C1)C2=CC(=O)C3=C(O2)C=C(C(=C3...,Fatty acid-binding protein 5 (FABP5),0,1,0
3544,WGNIVAMNAWBYRO-UHFFFAOYSA-N,MDQNQHLNKTAEAQPSENKKTRYCNGLKMFLAALSLSFIAKTLGAI...,Gancaonin Q,CC(=CCC1=C(C=CC(=C1)C2=CC(=O)C3=C(O2)C=C(C(=C3...,Solute carrier organic anion transporter famil...,0,1,0
...,...,...,...,...,...,...,...,...
195835,UXYRZJKIQKRJCF-TZPFWLJSSA-N,MSNRKKVAYFHDPDIGSYYYGAGHPMKPQRIRMTHSLIVSYNLYKY...,Mesterolone,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Histone Deacetylase (pfHDAC-1),0,1,0
195836,UXYRZJKIQKRJCF-TZPFWLJSSA-N,MTSTGQDSSTRQRKSRHNPQSPLQESSATLKRGGKKCAVPHSSPNL...,Mesterolone,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Histone Deacetylase 6 (HDAC6),0,1,0
195837,UXYRZJKIQKRJCF-TZPFWLJSSA-N,MAIGYVWNTLYGWVDTGTGSLAAANLTARMQPISHHLAHPDTKRRF...,Mesterolone,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Histone deacetylase-like amidohydrolase,0,1,0
195838,UXYRZJKIQKRJCF-TZPFWLJSSA-N,SSPITGLVYDQRMMLHHNMWDSHHPELPQRISRIFSRHEELRLLSR...,Mesterolone,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Catalytic domain 2 (zCD2),0,1,0


In [110]:
ndm_all_neg = pd.concat([nonactive_smiles,ndm_bd_df_neg,shortest_path_df_13_non_binding])

In [111]:
ndm_all_neg = ndm_all_neg.drop_duplicates(subset=['InChiKey','aaseq'])

In [112]:
ndm_all_neg

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB
6531,NaN,AAOFJKLTRKOQTQ-UHFFFAOYSA-N,CC1=C(C(=C(C=C1OC)OC)C(=O)C)O,CYTOCHROME P450 3A4,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,0,1,2
6760,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,ESTROGEN RECEPTOR BETA,MDIKNSPSSLNSPSSYNCSQSILPLEHGSIYIPSSYVDSHHEYPAM...,0,1,2
6766,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,MAP KINASE P38 ALPHA,MSQERPTFYRQELNKTIWEVPERYQNLSPVGSGAYGSVCAAFDTKT...,0,1,2
6803,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,C-C CHEMOKINE RECEPTOR TYPE 5,MDYQVSSPIYDINYYTSEPCQKINVKQIAARLLPPLYSLVFIFGFV...,0,1,2
6805,DIAZEPAM,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3,C-C CHEMOKINE RECEPTOR TYPE 4,MNPTDIADTTLDESIYSNYYLYESIPKPCTKEGIKAFGELFLPPLY...,0,1,2
...,...,...,...,...,...,...,...,...
195835,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Histone Deacetylase (pfHDAC-1),MSNRKKVAYFHDPDIGSYYYGAGHPMKPQRIRMTHSLIVSYNLYKY...,0,1,0
195836,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Histone Deacetylase 6 (HDAC6),MTSTGQDSSTRQRKSRHNPQSPLQESSATLKRGGKKCAVPHSSPNL...,0,1,0
195837,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Histone deacetylase-like amidohydrolase,MAIGYVWNTLYGWVDTGTGSLAAANLTARMQPISHHLAHPDTKRRF...,0,1,0
195838,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Catalytic domain 2 (zCD2),SSPITGLVYDQRMMLHHNMWDSHHPELPQRISRIFSRHEELRLLSR...,0,1,0


In [114]:
ndm_all_neg[ndm_all_neg['is_bindingDB'] != 0].to_csv('absolute_NDM_negative_samples.csv')

In [206]:
ndm_all_neg.to_pickle('NDM_Negative_Samples_Clean.pkl')

# Combining into single dataframe

In [207]:
db_pos = pd.read_pickle('DrugBank_Positive_Samples_Clean.pkl')
db_neg = pd.read_pickle('DrugBank_Negative_Samples_Clean.pkl')
ndm_pos = pd.read_pickle('NDM_Positive_Samples_Clean.pkl')
ndm_neg = pd.read_pickle('NDM_Negative_Samples_Clean.pkl')

In [208]:
final_df = pd.concat([db_pos,db_neg,ndm_pos,ndm_neg])

In [209]:
final_df = final_df.drop_duplicates(subset=['InChiKey','aaseq'])

In [210]:
final_df

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB
30,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Prothrombin,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...,1,0,0
31,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Myeloperoxidase,MGVPFFSSLRCMVDLGPCWAGGLTAEMKLLLALAGLLAILATPQPS...,1,0,0
32,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Gonadotropin-releasing hormone receptor,MANSASPEQNQNHCSAINNSIPLMQGNLPTLTLSGKIRVTVTFFLF...,1,0,0
33,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Cytochrome P450 3A4,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,1,0,0
56,Goserelin,BLCLNMBMMGCOAS-URPVMXJPSA-N,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,Lutropin-choriogonadotropic hormone receptor,MKQRFSALQLLKLLLLLQPPLPRALREALCPEPCNCVPDGALRCPG...,1,0,0
...,...,...,...,...,...,...,...,...
195835,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,NaN,MSNRKKVAYFHDPDIGSYYYGAGHPMKPQRIRMTHSLIVSYNLYKY...,0,1,0
195836,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Histone deacetylase 6,MTSTGQDSSTRQRKSRHNPQSPLQESSATLKRGGKKCAVPHSSPNL...,0,1,0
195837,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Histone deacetylase-like amidohydrolase,MAIGYVWNTLYGWVDTGTGSLAAANLTARMQPISHHLAHPDTKRRF...,0,1,0
195838,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,NaN,SSPITGLVYDQRMMLHHNMWDSHHPELPQRISRIFSRHEELRLLSR...,0,1,0


In [211]:
final_df.to_pickle('DrugBank_NDM_Samples_Master.pkl')

# Adding fingerprints

In [214]:
final_df = pd.read_pickle('DrugBank_NDM_Samples_Master.pkl')

In [216]:
final_df

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB,fingerprint
30,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Prothrombin,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...,1,0,0,2246728737 3542456614 2245384272 1173125914 22...
31,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Myeloperoxidase,MGVPFFSSLRCMVDLGPCWAGGLTAEMKLLLALAGLLAILATPQPS...,1,0,0,847336149 4202572743 864942730 2380084179 8649...
32,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Gonadotropin-releasing hormone receptor,MANSASPEQNQNHCSAINNSIPLMQGNLPTLTLSGKIRVTVTFFLF...,1,0,0,2245384272 2064745600 2246699815 4278941385 86...
33,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Cytochrome P450 3A4,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,1,0,0,None
56,Goserelin,BLCLNMBMMGCOAS-URPVMXJPSA-N,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,Lutropin-choriogonadotropic hormone receptor,MKQRFSALQLLKLLLLLQPPLPRALREALCPEPCNCVPDGALRCPG...,1,0,0,2245384272 1510461303 2245384272 1167322652 22...
...,...,...,...,...,...,...,...,...,...
195835,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,NaN,MSNRKKVAYFHDPDIGSYYYGAGHPMKPQRIRMTHSLIVSYNLYKY...,0,1,0,2246728737 1858577693 2976033787 1236795370 29...
195836,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Histone deacetylase 6,MTSTGQDSSTRQRKSRHNPQSPLQESSATLKRGGKKCAVPHSSPNL...,0,1,0,2246728737 1858577693 2976033787 1236795370 29...
195837,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Histone deacetylase-like amidohydrolase,MAIGYVWNTLYGWVDTGTGSLAAANLTARMQPISHHLAHPDTKRRF...,0,1,0,2246728737 1858577693 2976033787 1236795370 29...
195838,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,NaN,SSPITGLVYDQRMMLHHNMWDSHHPELPQRISRIFSRHEELRLLSR...,0,1,0,2246728737 1858577693 2976033787 1236795370 29...


In [154]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs

In [155]:
def get_ordered_fingerprint_string(SMILE, radius=1):
    bi = {}
    AllChem.GetMorganFingerprint(Chem.MolFromSmiles(SMILE), radius=radius, bitInfo=bi)
    mol = pd.DataFrame()
    for key, value in bi.items():
        for i in range(len(value)):
            sub = pd.Series()
            sub['val'] = str(key)
            sub['order'] = value[i][0]
            sub['radius'] = value[i][1]
            mol = mol.append(sub, ignore_index=True)
    mol = mol.sort_values(by=['order', 'radius']).reset_index(drop=True)
    # display(mol)
    mol_string = " ".join(mol.val.tolist())
    return mol_string.strip()

In [223]:
for index, row in final_df.iterrows():
    final_df.at[index,'fingerprint'] = get_ordered_fingerprint_string(str(row['chem_smile']),radius=2)

C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\an

C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\an

C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\an

C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\an

C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\an

...

C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\an

C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\an

C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\an

C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\Ayan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys


ArgumentError: Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprint(NoneType)
did not match C++ signature:
    GetMorganFingerprint(class RDKit::ROMol mol, int radius, class boost::python::api::object invariants=[], class boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, bool useCounts=True, class boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)

In [157]:
final_df

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB,fingerprint
30,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Prothrombin,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...,1,0,0,2246728737 3542456614 2245384272 1506563592 32...
31,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Myeloperoxidase,MGVPFFSSLRCMVDLGPCWAGGLTAEMKLLLALAGLLAILATPQPS...,1,0,0,2246728737 3542456614 2245384272 1506563592 32...
32,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Gonadotropin-releasing hormone receptor,MANSASPEQNQNHCSAINNSIPLMQGNLPTLTLSGKIRVTVTFFLF...,1,0,0,2246728737 3542456614 2245384272 773607102 129...
33,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Cytochrome P450 3A4,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,1,0,0,2246728737 3542456614 2245384272 773607102 129...
56,Goserelin,BLCLNMBMMGCOAS-URPVMXJPSA-N,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,Lutropin-choriogonadotropic hormone receptor,MKQRFSALQLLKLLLLLQPPLPRALREALCPEPCNCVPDGALRCPG...,1,0,0,2246728737 3537119515 2245273601 2423543607 23...
...,...,...,...,...,...,...,...,...,...
195835,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,NaN,MSNRKKVAYFHDPDIGSYYYGAGHPMKPQRIRMTHSLIVSYNLYKY...,0,1,0,NaN
195836,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Histone deacetylase 6,MTSTGQDSSTRQRKSRHNPQSPLQESSATLKRGGKKCAVPHSSPNL...,0,1,0,NaN
195837,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Histone deacetylase-like amidohydrolase,MAIGYVWNTLYGWVDTGTGSLAAANLTARMQPISHHLAHPDTKRRF...,0,1,0,NaN
195838,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,NaN,SSPITGLVYDQRMMLHHNMWDSHHPELPQRISRIFSRHEELRLLSR...,0,1,0,NaN


In [232]:
final_df_fp = final_df[~final_df['fingerprint'].isna()]

In [234]:
final_df_fp

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB,fingerprint
30,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Prothrombin,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...,1,0,0,2246728737 3542456614 2245384272 1506563592 32...
31,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,Myeloperoxidase,MGVPFFSSLRCMVDLGPCWAGGLTAEMKLLLALAGLLAILATPQPS...,1,0,0,2246728737 3542456614 2245384272 1506563592 32...
32,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Gonadotropin-releasing hormone receptor,MANSASPEQNQNHCSAINNSIPLMQGNLPTLTLSGKIRVTVTFFLF...,1,0,0,2246728737 3542456614 2245384272 773607102 129...
33,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,Cytochrome P450 3A4,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,1,0,0,2246728737 3542456614 2245384272 773607102 129...
56,Goserelin,BLCLNMBMMGCOAS-URPVMXJPSA-N,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,Lutropin-choriogonadotropic hormone receptor,MKQRFSALQLLKLLLLLQPPLPRALREALCPEPCNCVPDGALRCPG...,1,0,0,2246728737 3537119515 2245273601 2423543607 23...
...,...,...,...,...,...,...,...,...,...
195835,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,NaN,MSNRKKVAYFHDPDIGSYYYGAGHPMKPQRIRMTHSLIVSYNLYKY...,0,1,0,2246728737 1858577693 2976033787 1236795370 29...
195836,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Histone deacetylase 6,MTSTGQDSSTRQRKSRHNPQSPLQESSATLKRGGKKCAVPHSSPNL...,0,1,0,2246728737 1858577693 2976033787 1236795370 29...
195837,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,Histone deacetylase-like amidohydrolase,MAIGYVWNTLYGWVDTGTGSLAAANLTARMQPISHHLAHPDTKRRF...,0,1,0,2246728737 1858577693 2976033787 1236795370 29...
195838,Mesterolone,UXYRZJKIQKRJCF-TZPFWLJSSA-N,C[C@H]1CC(=O)C[C@H]2[C@]1([C@H]3CC[C@]4([C@H](...,NaN,SSPITGLVYDQRMMLHHNMWDSHHPELPQRISRIFSRHEELRLLSR...,0,1,0,2246728737 1858577693 2976033787 1236795370 29...


In [237]:
final_df['fingerprint'].isna().sum()

10959

In [238]:
final_df_fp.to_pickle('DrugBank_NDM_Samples_Master_w_fp.pkl')

In [239]:
final_df_no_fp = final_df[final_df['fingerprint'].isna()]
len(set(final_df_no_fp['InChiKey'].tolist()))

1650

In [240]:
final_df_no_fp

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB,fingerprint
4399,Adinazolam,GJSLOMWRLALDCT-UHFFFAOYSA-N,CN(C)CC1=NN=C2CN=C(C3=CC=CC=C3)C3=C(C=CC(Cl)=C...,Gamma-aminobutyric acid receptor subunit delta,MDAPARLLAPLLLLCAQQLRGTRAMNDIGDYVGSNLEISWLPNLDG...,1,0,0,None
4471,Bosentan,GJPICJJJRGTNOD-UHFFFAOYSA-N,COC1=CC=CC=C1OC1=C(NS(=O)(=O)C2=CC=C(C=C2)C(C)...,Bile salt export pump,MSDSVILRSIKKFGEENDGFESDKSYNNDKKSRLQDEKKGDGVRVG...,1,0,0,None
4472,Tigecycline,FPZLLRFZJZRHSY-HJYUBDRYSA-N,[H][C@@]12CC3=C(C(O)=C(NC(=O)CNC(C)(C)C)C=C3N(...,30S ribosomal protein S9,MAENQYYGTGRRKSSAARVFIKPGNGKIVINQRSLEQYFGRETARM...,1,0,0,None
4531,Carbamazepine,FFGPTBGBLSHEPO-UHFFFAOYSA-N,NC(=O)N1C2=CC=CC=C2C=CC2=CC=CC=C12,Cytochrome P450 3A5,MDLIPNLAVETWLLLAVSLVLLYLYGTRTHGLFKRLGIPGPTPLPL...,1,0,0,None
4890,Chloroquine,WHTVZRBIWZFKQO-UHFFFAOYSA-N,CCN(CC)CCCC(C)NC1=C2C=CC(Cl)=CC2=NC=C1,P-glycoprotein 1,MDLEGDRNGGAKKKNFFKLNNKSEKDKKEKKPTVSVFSMFRYSNWL...,1,0,0,None
...,...,...,...,...,...,...,...,...,...
5872239,COUMARIN,ZYGHJZDHTFUPRJ-UHFFFAOYSA-N,C1=CC=C2C(=C1)C=CC(=O)O2,CARBONIC ANHYDRASE II,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0,1,0,None
5872252,COUMARIN,ZYGHJZDHTFUPRJ-UHFFFAOYSA-N,C1=CC=C2C(=C1)C=CC(=O)O2,CARBONIC ANHYDRASE I,MASPDWGYDDKNGPEQWSKLYPIANGNNQSPVDIKTSETKHDTSLK...,0,1,0,None
5872256,COUMARIN,ZYGHJZDHTFUPRJ-UHFFFAOYSA-N,C1=CC=C2C(=C1)C=CC(=O)O2,EPIDERMAL GROWTH FACTOR RECEPTOR ERBB1,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,0,1,0,None
5872271,COUMARIN,ZYGHJZDHTFUPRJ-UHFFFAOYSA-N,C1=CC=C2C(=C1)C=CC(=O)O2,CARBONIC ANHYDRASE XII,MPRRSLHAAAVLLLVILKEQPSSPAPVNGSKWTYFGPDGENSWSKK...,0,1,0,None


In [241]:
final_df_no_fp.to_pickle('DrugBank_NDM_Samples_Master_without_fp.pkl')

## Set Analysis

In [223]:
len(set(db_bd_pairs).intersection(set(ndm_bd_pairs)))

1337

In [224]:
len(set(db_full_pairs).intersection(set(ndm_bd_pairs)))

275

In [225]:
len(set(db_bd_pairs).intersection(set(db_full_pairs).intersection(set(ndm_bd_pairs))))

211

In [226]:
len(set(db_full_pairs).difference(set(db_bd_pairs)).intersection(set(ndm_bd_pairs)))

64

In [233]:
mismatch_list = list(set(db_full_pairs).difference(set(db_bd_pairs)).intersection(set(ndm_bd_pairs)))
chem_mismatch_inchi = []
 
for pair in mismatch_list:
    chem_mismatch_inchi.append(pair[0])
    
len(set(chem_mismatch_inchi))

26

In [237]:
to_drop_mismatch = []

for index, row in bd_load.iterrows():
    if row['Ligand InChI Key'] not in chem_mismatch_inchi:
        to_drop_mismatch.append(index)

bd_load_mismatch = bd_load.drop(to_drop_mismatch)

In [238]:
bd_load_mismatch

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name Assigned by Curator or DataSource,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,UniProt (SwissProt) Recommended Name of Target Chain.12,UniProt (SwissProt) Entry Name of Target Chain.12,UniProt (SwissProt) Primary ID of Target Chain.12,UniProt (SwissProt) Secondary ID(s) of Target Chain.12,UniProt (SwissProt) Alternative ID(s) of Target Chain.12,UniProt (TrEMBL) Submitted Name of Target Chain.12,UniProt (TrEMBL) Entry Name of Target Chain.12,UniProt (TrEMBL) Primary ID of Target Chain.12,UniProt (TrEMBL) Secondary ID(s) of Target Chain.12,UniProt (TrEMBL) Alternative ID(s) of Target Chain.12
18370,18370,Cn1c2nc[nH]c2c(=O)n(C)c1=O,InChI=1S/C7H8N4O2/c1-10-5-4(8-3-9-5)6(12)11(2)...,ZFXYFBGIUFBOJW-UHFFFAOYSA-N,10847,"1,3-dimethyl-2,3,6,7-tetrahydro-1H-purine-2,6-...",Endochitinase B1,Aspergillus fumigatiaffinis,NaN,1500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18373,18373,Cn1c2nc[nH]c2c(=O)n(C)c1=O,InChI=1S/C7H8N4O2/c1-10-5-4(8-3-9-5)6(12)11(2)...,ZFXYFBGIUFBOJW-UHFFFAOYSA-N,10847,"1,3-dimethyl-2,3,6,7-tetrahydro-1H-purine-2,6-...","cAMP-specific 3',5'-cyclic phosphodiesterase 4A",Homo sapiens,NaN,386000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18376,18376,Cn1c2nc[nH]c2c(=O)n(C)c1=O,InChI=1S/C7H8N4O2/c1-10-5-4(8-3-9-5)6(12)11(2)...,ZFXYFBGIUFBOJW-UHFFFAOYSA-N,10847,"1,3-dimethyl-2,3,6,7-tetrahydro-1H-purine-2,6-...",Chitinase,Homo sapiens,NaN,>500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53742,53745,NCCc1c[nH]c2ccc(O)cc12,InChI=1S/C10H12N2O/c11-4-3-7-6-12-10-2-1-8(13)...,QZAYGJVTTNCVMB-UHFFFAOYSA-N,10755,14C-5-hydroxy tryptamine creatinine disulfate:...,5-hydroxytryptamine receptor 4 (5-HT4),NaN,1049,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64677,64680,C\C(\C=C\C1=C(C)CCCC1(C)C)=C/C=C/C(/C)=C/C(O)=O,InChI=1S/C20H28O2/c1-15(8-6-9-16(2)14-19(21)22...,SHGAZHPCJJPHSC-YCNIQYBTSA-N,31883,9-cis-retinoic acid (9cRA)::ALL-TRANS-RETINOIC...,Retinoic acid receptor RXR-alpha,Homo sapiens,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78568,78571,Oc1cc2c3c(oc(=O)c4cc(O)c(O)c(oc2=O)c34)c1O,InChI=1S/C14H6O8/c15-5-1-3-7-8-4(14(20)22-11(7...,AFSDNFLWKVMVRB-UHFFFAOYSA-N,4078,"6,7,13,14-tetrahydroxy-2,9-dioxatetracyclo[6.6...",acetyl-CoA acetyltransferase/HMG-CoA reductase,Enterococcus faecalis,NaN,1265390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146771,146774,COc1cc(C=CC(=O)CC(=O)C=Cc2ccc(O)c(OC)c2)ccc1O,InChI=1S/C21H20O6/c1-26-20-11-14(5-9-18(20)24)...,VFLDPWHFBUODDF-FCXRPNKRSA-N,29532,"(1E,6E)-1,7-bis(3-methoxy-4-oxidanyl-phenyl)he...",perilipin-1,Homo sapiens,NaN,9167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146816,146819,COc1cc(C=CC(=O)CC(=O)C=Cc2ccc(O)c(OC)c2)ccc1O,InChI=1S/C21H20O6/c1-26-20-11-14(5-9-18(20)24)...,VFLDPWHFBUODDF-FCXRPNKRSA-N,29532,"(1E,6E)-1,7-bis(3-methoxy-4-oxidanyl-phenyl)he...",perilipin-5,Homo sapiens,NaN,10232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155278,155281,NCCc1c[nH]c2ccc(O)cc12,InChI=1S/C10H12N2O/c11-4-3-7-6-12-10-2-1-8(13)...,QZAYGJVTTNCVMB-UHFFFAOYSA-N,10755,14C-5-hydroxy tryptamine creatinine disulfate:...,5-HT5,Mus musculus,251.18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158186,158189,NCCc1c[nH]c2ccc(O)cc12,InChI=1S/C10H12N2O/c11-4-3-7-6-12-10-2-1-8(13)...,QZAYGJVTTNCVMB-UHFFFAOYSA-N,10755,14C-5-hydroxy tryptamine creatinine disulfate:...,DRD1,Canis familiaris,>10000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [241]:
list(bd_load_mismatch.columns)

['BindingDB Reactant_set_id',
 'Ligand SMILES',
 'Ligand InChI',
 'Ligand InChI Key',
 'BindingDB MonomerID',
 'BindingDB Ligand Name',
 'Target Name Assigned by Curator or DataSource',
 'Target Source Organism According to Curator or DataSource',
 'Ki (nM)',
 'IC50 (nM)',
 'Kd (nM)',
 'EC50 (nM)',
 'kon (M-1-s-1)',
 'koff (s-1)',
 'pH',
 'Temp (C)',
 'Curation/DataSource',
 'Article DOI',
 'PMID',
 'PubChem AID',
 'Patent Number',
 'Authors',
 'Institution',
 'Link to Ligand in BindingDB',
 'Link to Target in BindingDB',
 'Link to Ligand-Target Pair in BindingDB',
 'Ligand HET ID in PDB',
 'PDB ID(s) for Ligand-Target Complex',
 'PubChem CID',
 'PubChem SID',
 'ChEBI ID of Ligand',
 'ChEMBL ID of Ligand',
 'DrugBank ID of Ligand',
 'IUPHAR_GRAC ID of Ligand',
 'KEGG ID of Ligand',
 'ZINC ID of Ligand',
 'Number of Protein Chains in Target (>1 implies a multichain complex)',
 'BindingDB Target Chain  Sequence',
 'PDB ID(s) of Target Chain',
 'UniProt (SwissProt) Recommended Name of Tar

In [242]:
bd_load_mismatch['DrugBank ID of Ligand'].tolist()

['DB00277',
 'DB00277',
 'DB00277',
 nan,
 nan,
 'DB08468',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'DB00667',
 nan,
 'DB00667',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'DB08468',
 'DB08468',
 'DB08468',
 nan,
 'DB08468',
 nan,
 'DB07690']

# Extra Negative Samples

In [115]:
drugbank_bd_df_non_binding = pd.read_csv('drugbank_bd_df_non_binding.csv')
absolute_NDM_negative_samples = pd.read_csv('absolute_NDM_negative_samples.csv')

In [116]:
aboslute_negatives = pd.concat([drugbank_bd_df_non_binding,absolute_NDM_negative_samples])

In [119]:
absolute_negative.to_csv('absolute_negatives_bindingDB_DTC.csv')

In [6]:
drugbank_all_neg = pd.read_pickle('DrugBank_Negative_Samples_Clean.pkl')

In [8]:
drugbank_all_neg[drugbank_all_neg['is_bindingDB'] == 1]

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB
0,Astemizole,GXDALQBWZGODGZ-UHFFFAOYSA-N,COC1=CC=C(CCN2CCC(CC2)NC2=NC3=CC=CC=C3N2CC2=CC...,Heparanase,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,0,0,1
2,Guvacine,QTDZOWFRBNTPQR-UHFFFAOYSA-N,OC(=O)C1=CCCNC1,Sodium- and chloride-dependent betaine transpo...,MDGKVAVQECGPPAVSWVPEEGEKLDQEDEDQVKDRGQWTNKMEFV...,0,0,1
4,Halazepam,WYCLKVQLVUQKNZ-UHFFFAOYSA-N,FC(F)(F)CN1C2=C(C=C(Cl)C=C2)C(=NCC1=O)C1=CC=CC=C1,Translocator protein,MAPPWVPAMGFTLAPSLGCFVGSRFVHGEGLRWYAGLQKPSWHPPH...,0,0,1
6,Clonazepam,DGBIGWXXNGSACT-UHFFFAOYSA-N,[O-][N+](=O)C1=CC2=C(NC(=O)CN=C2C2=CC=CC=C2Cl)...,Translocator protein,MAPPWVPAMGFTLAPSLGCFVGSRFVHGEGLRWYAGLQKPSWHPPH...,0,0,1
7,Salicylic acid,YGSDEFSMJLZEOE-UHFFFAOYSA-N,OC(=O)C1=CC=CC=C1O,Solute carrier family 22 member 8,MTFSEILDRVGSMGHFQFLHVAILGLPILNMANHNLLQIFTAATPV...,0,0,1
...,...,...,...,...,...,...,...,...
385,Diazepam,AAOVKJBEBIDNHE-UHFFFAOYSA-N,CN1C2=C(C=C(Cl)C=C2)C(=NCC1=O)C1=CC=CC=C1,Translocator protein,MAPPWVPAMGFTLAPSLGCFVGSRFVHGEGLRWYAGLQKPSWHPPH...,0,0,1
387,Acetohydroxamic acid,RRUDCFGSUDOHDG-UHFFFAOYSA-N,CC(=O)NO,72 kDa type IV collagenase,MEALMARGALTGPLRALCLLGCLLSHAAAAPSPIIKFPGDVAPKTD...,0,0,1
389,Methyldopa,CJCSPKMFHVPWAR-JTQLQIEISA-N,C[C@](N)(CC1=CC=C(O)C(O)=C1)C(O)=O,Cystathionine gamma-lyase,MQEKDASSQGFLPHFQHFATQAIHVGQDPEQWTSRAVVPPISLSTT...,0,0,1
391,S-METHYL-GLUTATHIONE,QTQDDTSVRVWHMO-BQBZGAKWSA-N,CSC[C@H](NC(=O)CC[C@H](N)C(O)=O)C(=O)NCC(O)=O,Lactoylglutathione lyase,MAEPQPPSGGLTDEAALSCCSDADPSTKDFLLQQTMLRVKDPKKSL...,0,0,1


In [9]:
ndm_all_neg = pd.read_pickle('NDM_Negative_Samples_Clean.pkl')

In [11]:
ndm_all_neg[ndm_all_neg['is_bindingDB'] == 1]

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB
0,Fumaric acid,VZCYOOQTPOCHFL-OWOJBTEDSA-N,C(=C/C(=O)O)\C(=O)O,Solute carrier family 22 member 20,MAFTDLLDALGGVGRFQLVYTALLLLPCGLLACHTFLQNFTAAAPP...,0,1,1
1,Thiourea,UMGDCJDMYOKAJW-UHFFFAOYSA-N,C(=N)(N)S,Alcohol dehydrogenase E chain,MSTAGKVIKCKAAVLWEEKKPFSIEEVEVAPPKAHEVRIKMVATGI...,0,1,1
2,Chloride,VEXZGXHMUGYJMC-UHFFFAOYSA-M,[Cl-],NaN,MRFVSMIIKDILRENQDFRFRDLSDLKHSPKLCIITCMDSRLIDLL...,0,1,1
3,Baicalein,FXNFHKRTJBSTCS-UHFFFAOYSA-N,C1=CC=C(C=C1)C2=CC(=O)C3=C(O2)C=C(C(=C3O)O)O,ABAT,MASMLLAQRLACSFQHSYRLLVPGSRHISQAAAKVDVEFDYDGPLM...,0,1,1
4,Dimethyl sulfone,HHVIBTZHLRERCL-UHFFFAOYSA-N,CS(=O)(=O)C,Protein cereblon,MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNI...,0,1,1
...,...,...,...,...,...,...,...,...
481,demethylsuberosin,FIDUIAPDSKSUGO-UHFFFAOYSA-N,CC(=CCC1=C(C=C2C(=C1)C=CC(=O)O2)O)C,ACETYLCHOLINESTERASE,MRPPQCLLHTPSLASPLLLLLLWLLGGGVGAEGREDAELLVTVRGG...,0,1,1
482,Thiophen-3-carboxyaldehyde,RBIGKSZIQCTIJF-UHFFFAOYSA-N,C1=CSC=C1C=O,Protein kinase C zeta type,MPSRTDPKMDRSGGRVRLKAHYGGDILITSVDPTTTFQDLCEEVRD...,0,1,1
483,Iodide,XMBWDFGMSWQBCA-UHFFFAOYSA-M,[I-],NaN,MRFVSMIIKDILRENQDFRFRDLSDLKHSPKLCIITCMDSRLIDLL...,0,1,1
484,S-methylglutathione,QTQDDTSVRVWHMO-BQBZGAKWSA-N,CSC[C@@H](C(=O)NCC(=O)O)NC(=O)CC[C@@H](C(=O)O)N,GLO1,MAEPQPPSGGLTDEAALSCCSDADPSTKDFLLQQTMLRVKDPKKSL...,0,1,1


In [12]:
absolute_negative = pd.concat([drugbank_all_neg[drugbank_all_neg['is_bindingDB'] == 1],ndm_all_neg[ndm_all_neg['is_bindingDB'] == 1]])

In [13]:
absolute_negative

,chem_name,InChiKey,chem_smile,target_name,aaseq,Y,is_ndm,is_bindingDB
0,Astemizole,GXDALQBWZGODGZ-UHFFFAOYSA-N,COC1=CC=C(CCN2CCC(CC2)NC2=NC3=CC=CC=C3N2CC2=CC...,Heparanase,MLLRSKPALPPPLMLLLLGPLGPLSPGALPRPAQAQDVVDLDFFTQ...,0,0,1
2,Guvacine,QTDZOWFRBNTPQR-UHFFFAOYSA-N,OC(=O)C1=CCCNC1,Sodium- and chloride-dependent betaine transpo...,MDGKVAVQECGPPAVSWVPEEGEKLDQEDEDQVKDRGQWTNKMEFV...,0,0,1
4,Halazepam,WYCLKVQLVUQKNZ-UHFFFAOYSA-N,FC(F)(F)CN1C2=C(C=C(Cl)C=C2)C(=NCC1=O)C1=CC=CC=C1,Translocator protein,MAPPWVPAMGFTLAPSLGCFVGSRFVHGEGLRWYAGLQKPSWHPPH...,0,0,1
6,Clonazepam,DGBIGWXXNGSACT-UHFFFAOYSA-N,[O-][N+](=O)C1=CC2=C(NC(=O)CN=C2C2=CC=CC=C2Cl)...,Translocator protein,MAPPWVPAMGFTLAPSLGCFVGSRFVHGEGLRWYAGLQKPSWHPPH...,0,0,1
7,Salicylic acid,YGSDEFSMJLZEOE-UHFFFAOYSA-N,OC(=O)C1=CC=CC=C1O,Solute carrier family 22 member 8,MTFSEILDRVGSMGHFQFLHVAILGLPILNMANHNLLQIFTAATPV...,0,0,1
...,...,...,...,...,...,...,...,...
481,demethylsuberosin,FIDUIAPDSKSUGO-UHFFFAOYSA-N,CC(=CCC1=C(C=C2C(=C1)C=CC(=O)O2)O)C,ACETYLCHOLINESTERASE,MRPPQCLLHTPSLASPLLLLLLWLLGGGVGAEGREDAELLVTVRGG...,0,1,1
482,Thiophen-3-carboxyaldehyde,RBIGKSZIQCTIJF-UHFFFAOYSA-N,C1=CSC=C1C=O,Protein kinase C zeta type,MPSRTDPKMDRSGGRVRLKAHYGGDILITSVDPTTTFQDLCEEVRD...,0,1,1
483,Iodide,XMBWDFGMSWQBCA-UHFFFAOYSA-M,[I-],NaN,MRFVSMIIKDILRENQDFRFRDLSDLKHSPKLCIITCMDSRLIDLL...,0,1,1
484,S-methylglutathione,QTQDDTSVRVWHMO-BQBZGAKWSA-N,CSC[C@@H](C(=O)NCC(=O)O)NC(=O)CC[C@@H](C(=O)O)N,GLO1,MAEPQPPSGGLTDEAALSCCSDADPSTKDFLLQQTMLRVKDPKKSL...,0,1,1


In [14]:
absolute_negative.to_csv('absolute_negatives_bindingDB.csv')